In [8]:
# -*- coding: utf-8 -*-
"""
Unified CPI Forecasting Script (v2 - Fixed ML/DL/Ensemble Alignment)

This script runs multiple forecasting models on CPI data and saves the results.
It includes traditional statistical models, machine learning models, deep learning models,
and hybrid/ensemble approaches.

Fixes in v2:
- Corrected feature alignment for ML models.
- Addressed input shape issues for DL models.
- Fixed index alignment for Boosting Ensemble.
- Revised Stacking Ensemble training data generation.
"""

import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import logging
import time
import warnings
import json
from datetime import datetime
from typing import Dict, List, Tuple, Optional, Any, Union

# Suppress warnings
warnings.filterwarnings("ignore")

# Import required libraries for models
import statsmodels.api as sm
from statsmodels.tsa.statespace.sarimax import SARIMAX
from statsmodels.tsa.holtwinters import ExponentialSmoothing
from statsmodels.tsa.stattools import adfuller, kpss
from statsmodels.tsa.seasonal import seasonal_decompose
from statsmodels.tsa.api import VAR, ETSModel
from statsmodels.tsa.forecasting.theta import ThetaModel

from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.metrics import mean_squared_error, mean_absolute_error, mean_absolute_percentage_error
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor
from sklearn.linear_model import LinearRegression, ElasticNet, Ridge, RidgeCV
from sklearn.svm import SVR
from sklearn.neighbors import KNeighborsRegressor
from sklearn.feature_selection import RFE

import xgboost as xgb
import lightgbm as lgb

try:
    from prophet import Prophet
    prophet_available = True
except ImportError:
    prophet_available = False
    print("Prophet not available. Prophet models will be skipped.")

try:
    import torch
    import torch.nn as nn
    from torch.utils.data import TensorDataset, DataLoader
    torch_available = True
except ImportError:
    torch_available = False
    print("PyTorch not available. Deep learning models will be skipped.")

try:
    import optuna
    optuna_available = True
    optuna.logging.set_verbosity(optuna.logging.WARNING)
except ImportError:
    optuna_available = False
    print("Optuna not available. Hyperparameter optimization will be disabled.")

# Setup logging
os.makedirs("cpi_forecast_results", exist_ok=True)
logging.basicConfig(
    level=logging.INFO,
    format="%(asctime)s - %(levelname)s - %(message)s",
    handlers=[
        logging.FileHandler("cpi_forecast_results/forecast_log.txt", mode="w"),
        logging.StreamHandler()
    ]
)
logger = logging.getLogger(__name__)

# Configuration
CONFIG = {
    "forecast_horizon": 12,       # Number of steps to forecast ahead
    "test_months": 12,            # Number of months for testing
    "seasonal_periods": 12,       # Monthly data
    "results_dir": "cpi_forecast_results",
    "random_seed": 42,            # For reproducibility
    "n_jobs": -1,                 # Use all available cores
    "optuna_trials": 20,          # Number of trials for hyperparameter optimization
    "dl_sequence_length": 12,     # Sequence length for deep learning models
    "dl_epochs": 100,              # Number of epochs for deep learning models
    "dl_batch_size": 32,          # Batch size for deep learning models
    "dl_patience": 5,             # Early stopping patience
    "lag_features": list(range(1,13)), # Lag features to create
    "rolling_windows": [3, 6, 12], # Rolling window sizes for features
    "enable_traditional_models": True,
    "enable_ml_models": True,
    "enable_dl_models": True,
    "enable_hybrid_models": True,
    "enable_ensemble_models": True,
    "save_plots": True,
    "save_forecasts": True,
    "save_metrics": True,
    "verbose": True
}

# Set random seeds for reproducibility
np.random.seed(CONFIG['random_seed'])
if torch_available:
    torch.manual_seed(CONFIG['random_seed'])
    if torch.cuda.is_available():
        torch.cuda.manual_seed(CONFIG['random_seed'])
        torch.backends.cudnn.deterministic = True

# ================ UTILITY FUNCTIONS ================

def load_data(file_path: str) -> pd.DataFrame:
    """Load the preprocessed CPI data."""
    logger.info(f"Loading data from {file_path}")
    try:
        df = pd.read_csv(file_path, index_col=0, parse_dates=True)
        if not isinstance(df.index, pd.DatetimeIndex):
            raise ValueError("DataFrame index must be a DatetimeIndex")
        if 'cpi' not in df.columns:
            raise ValueError("DataFrame must contain 'cpi' column")
        logger.info(f"Data loaded. Shape: {df.shape}")
        logger.info(f"Date range: {df.index.min()} to {df.index.max()}")
        return df
    except Exception as e:
        logger.error(f"Failed to load data: {e}")
        raise


def train_test_split_ts(df: pd.DataFrame, test_months: int = 12) -> Tuple[pd.DataFrame, pd.DataFrame]:
    """Split time series data into train and test sets based on last N months."""
    if not isinstance(df.index, pd.DatetimeIndex):
        raise ValueError("DataFrame index must be a DatetimeIndex")
    
    # Calculate the cutoff date for the last test_months
    end_date = df.index.max()
    cutoff_date = end_date - pd.offsets.MonthEnd(test_months) + pd.offsets.MonthBegin(1)
    
    # Split data
    train = df[df.index < cutoff_date]
    test = df[df.index >= cutoff_date]
    
    logger.info(f"Train set: {train.shape}, Test set: {test.shape}")
    logger.info(f"Train period: {train.index.min()} to {train.index.max()}")
    logger.info(f"Test period: {test.index.min()} to {test.index.max()}")
    return train, test

def create_features(df: pd.DataFrame, target_col: str = "cpi") -> pd.DataFrame:
    """Tạo đặc trưng chuỗi thời gian."""
    logger.info("Creating features with enhanced feature engineering...")
    if target_col not in df.columns:
        logger.error(f"Target column '{target_col}' not found in DataFrame")
        return pd.DataFrame()
    df_features = df.copy()
    
    # Lag features
    for lag in CONFIG["lag_features"]:
        df_features[f"{target_col}_lag_{lag}"] = df_features[target_col].shift(lag)
    
    # Rolling window features
    for window in CONFIG["rolling_windows"]:
        df_features[f"{target_col}_roll_mean_{window}"] = df_features[target_col].rolling(window=window, min_periods=1).mean()
        df_features[f"{target_col}_roll_std_{window}"] = df_features[target_col].rolling(window=window, min_periods=1).std()
    
    # Lagged differences
    for lag in [1, 2, 3]:
        df_features[f"{target_col}_diff_{lag}"] = df_features[target_col].diff(lag)
    
    # Date-based features
    df_features["month"] = df_features.index.month
    df_features["quarter"] = df_features.index.quarter
    df_features["year"] = df_features.index.year
    
    # Cyclical encoding of month and quarter
    df_features["month_sin"] = np.sin(2 * np.pi * df_features.index.month / 12)
    df_features["month_cos"] = np.cos(2 * np.pi * df_features.index.month / 12)
    df_features["quarter_sin"] = np.sin(2 * np.pi * df_features.index.quarter / 4)
    df_features["quarter_cos"] = np.cos(2 * np.pi * df_features.index.quarter / 4)
    
    # Fourier terms for seasonality
    for period in [6, 12, 24]:  # Thêm các chu kỳ 6, 12, 24 tháng
        df_features[f"fourier_sin_{period}"] = np.sin(2 * np.pi * np.arange(len(df_features)) / period)
        df_features[f"fourier_cos_{period}"] = np.cos(2 * np.pi * np.arange(len(df_features)) / period)
    
    # Trend feature
    df_features["time_idx"] = np.arange(len(df_features))
    
    # Drop target column
    df_features = df_features.drop(columns=[target_col], errors='ignore')
    
    # Fill NaNs
    df_features = df_features.fillna(method='bfill').fillna(method='ffill').fillna(0)
    
    logger.info(f"Features created. Shape: {df_features.shape}")
    return df_features

def calculate_metrics(actual: Union[pd.Series, np.ndarray], predicted: Union[pd.Series, np.ndarray], 
                     train_actual: Optional[Union[pd.Series, np.ndarray]] = None) -> Dict[str, float]:
    """Tính các chỉ số đánh giá dự báo, bao gồm Theil's U."""
    actual = np.array(actual)
    predicted = np.array(predicted)
    
    valid_mask = ~np.isnan(actual) & ~np.isnan(predicted) & ~np.isinf(actual) & ~np.isinf(predicted)
    actual_valid = actual[valid_mask]
    predicted_valid = predicted[valid_mask]
    
    if len(actual_valid) == 0:
        logger.warning("No valid data points for metrics calculation after filtering NaN/inf.")
        return {"RMSE": np.nan, "MAE": np.nan, "MAPE": np.nan, "sMAPE": np.nan, "MASE": np.nan, "DirAcc": np.nan, "Theil_U": np.nan}
    
    metrics = {}
    metrics['RMSE'] = np.sqrt(mean_squared_error(actual_valid, predicted_valid))
    metrics['MAE'] = mean_absolute_error(actual_valid, predicted_valid)
    
    # MAPE
    abs_actual = np.abs(actual_valid)
    metrics['MAPE'] = np.mean(np.abs((actual_valid - predicted_valid) / (abs_actual + 1e-9))) * 100
    
    # sMAPE
    denominator = np.abs(actual_valid) + np.abs(predicted_valid)
    metrics['sMAPE'] = np.mean(2 * np.abs(predicted_valid - actual_valid) / (denominator + 1e-9)) * 100
    
    # MASE
    if train_actual is not None and len(train_actual) > 1:
        train_actual = np.array(train_actual, dtype=float).flatten()
        naive_error = np.mean(np.abs(train_actual[1:] - train_actual[:-1]))
        if naive_error > 1e-9:
            metrics['MASE'] = metrics['MAE'] / naive_error
        else:
            metrics['MASE'] = np.nan
    else:
        metrics['MASE'] = np.nan
    
    # Directional Accuracy
    if len(actual_valid) > 1:
        actual_diff = np.diff(actual_valid)
        predicted_diff = np.diff(predicted_valid)
        metrics['DirAcc'] = np.mean(np.sign(actual_diff) == np.sign(predicted_diff)) * 100
    else:
        metrics['DirAcc'] = np.nan
    
    # Theil's U statistic
    if train_actual is not None:
        naive_forecast = train_actual[-1] * np.ones_like(actual_valid)
        naive_rmse = np.sqrt(mean_squared_error(actual_valid, naive_forecast))
        if naive_rmse > 1e-9:
            metrics['Theil_U'] = metrics['RMSE'] / naive_rmse
        else:
            metrics['Theil_U'] = np.nan
    else:
        metrics['Theil_U'] = np.nan
    
    return metrics

def check_stationarity(series: pd.Series, max_diff: int = 2) -> int:
    """Check stationarity of a time series and return the required differencing order."""
    series_clean = series.dropna().replace([np.inf, -np.inf], np.nan).dropna()
    if len(series_clean) < 20:  # Need sufficient data for tests
        logger.warning(f"Data too short ({len(series_clean)}) for reliable stationarity tests. Assuming d=0.")
        return 0
    
    d = 0
    try:
        adf_result = adfuller(series_clean)
        kpss_result = kpss(series_clean, regression="c", nlags="auto")
        adf_p = adf_result[1]
        kpss_p = kpss_result[1]
        logger.info(f"Stationarity tests (d=0): ADF p-value={adf_p:.4f}, KPSS p-value={kpss_p:.4f}")
        if adf_p < 0.05 and kpss_p > 0.05:
            logger.info(f"Series is likely stationary at d=0.")
            return 0
    except Exception as e:
        logger.warning(f"Stationarity test failed for d=0: {e}")
    
    temp_series = series_clean.copy()
    for d in range(1, max_diff + 1):
        temp_series = temp_series.diff().dropna()
        if len(temp_series) < 20:
            logger.warning(f"Data too short after differencing {d} times. Using previous d={d-1}.")
            return d - 1
        try:
            adf_result = adfuller(temp_series)
            kpss_result = kpss(temp_series, regression="c", nlags="auto")
            adf_p = adf_result[1]
            kpss_p = kpss_result[1]
            logger.info(f"Stationarity tests (d={d}): ADF p-value={adf_p:.4f}, KPSS p-value={kpss_p:.4f}")
            if adf_p < 0.05 and kpss_p > 0.05:
                logger.info(f"Series is likely stationary at differencing order d={d}.")
                return d
        except Exception as e:
            logger.warning(f"Stationarity test failed for d={d}: {e}")
    
    logger.warning(f"Series may not be stationary after {max_diff} differencing(s). Using d={max_diff}.")
    return max_diff

def plot_forecast(train: pd.Series, test: pd.Series, forecast: pd.Series, 
                 model_name: str, save_dir: str = None):
    """Plot the forecast results against actual values (zoom vào 12 điểm cuối train)."""
    plt.figure(figsize=(12, 6))
    
    # Chỉ lấy 12 điểm cuối của train
    train_tail = train[-12:]
    
    plt.plot(train_tail.index, train_tail, label="Train")
    plt.plot(test.index, test, label="Test (Actual)")
    plt.plot(forecast.index, forecast, label=f"Forecast ({model_name})", linestyle="--")
    
    plt.title(f"CPI Forecast: {model_name}")
    plt.xlabel("Date")
    plt.ylabel("CPI")
    plt.legend()
    plt.grid(True, alpha=0.3)
    plt.tight_layout()
    
    if save_dir:
        os.makedirs(save_dir, exist_ok=True)
        plt.savefig(f"{save_dir}/{model_name}_forecast.png", dpi=300)
        plt.close()
    else:
        plt.show()

def plot_comparison(train: pd.Series, test: pd.Series, forecasts: Dict[str, pd.Series], 
                   metrics: pd.DataFrame, save_dir: str = None):
    """Plot comparison of multiple forecasts (zoom vào 12 điểm cuối train)."""
    plt.figure(figsize=(14, 8))
    
    # Chỉ lấy 12 điểm cuối của train
    train_tail = train[-12:]
    
    plt.plot(train_tail.index, train_tail, label="Train", color="black", alpha=0.7)
    plt.plot(test.index, test, label="Test (Actual)", color="blue", linewidth=2)
    
    # Sort models by RMSE for better color assignment
    sorted_models = metrics.sort_values("RMSE").index
    colors = plt.cm.tab10(np.linspace(0, 1, len(forecasts)))
    color_dict = {model: colors[i] for i, model in enumerate(sorted_models) if model in forecasts}
    
    for model_name, forecast in forecasts.items():
        if forecast is None: continue  # Skip if forecast is None
        rmse = metrics.loc[model_name, "RMSE"] if model_name in metrics.index else None
        label = f"{model_name} (RMSE: {rmse:.3f})" if rmse else model_name
        color = color_dict.get(model_name, "gray")
        plt.plot(forecast.index, forecast, label=label, linestyle="--", alpha=0.8, color=color)
    
    plt.title("CPI Forecast Comparison")
    plt.xlabel("Date")
    plt.ylabel("CPI")
    plt.legend(loc="upper left", bbox_to_anchor=(1, 1))
    plt.grid(True, alpha=0.3)
    plt.tight_layout(rect=[0, 0, 0.85, 1])
    
    if save_dir:
        os.makedirs(save_dir, exist_ok=True)
        plt.savefig(f"{save_dir}/forecast_comparison.png", dpi=300)
        plt.close()
    else:
        plt.show()


def plot_metrics_comparison(metrics_df: pd.DataFrame, save_dir: str = None):
    """Plot comparison of model performance metrics."""
    metrics_to_plot = ["RMSE", "MAE", "MAPE", "sMAPE", "MASE", "DirAcc"]
    available_metrics = [m for m in metrics_to_plot if m in metrics_df.columns]
    
    n_metrics = len(available_metrics)
    n_cols = min(3, n_metrics)
    n_rows = (n_metrics + n_cols - 1) // n_cols
    
    fig, axes = plt.subplots(n_rows, n_cols, figsize=(5*n_cols, 4*n_rows))
    if n_metrics == 1:
        axes = np.array([axes])
    axes = axes.flatten()
    
    for i, metric in enumerate(available_metrics):
        # For DirAcc, higher is better, for others lower is better
        ascending = metric != "DirAcc"
        df_sorted = metrics_df.sort_values(by=metric, ascending=ascending)
        
        ax = axes[i]
        sns.barplot(x=df_sorted.index, y=df_sorted[metric], ax=ax)
        ax.set_title(f"{metric} Comparison")
        ax.set_xlabel("")
        ax.set_ylabel(metric)
        ax.tick_params(axis="x", rotation=45)
        ax.grid(axis="y", alpha=0.3)
    
    # Hide unused subplots
    for j in range(n_metrics, len(axes)):
        axes[j].set_visible(False)
    
    plt.tight_layout()
    
    if save_dir:
        os.makedirs(save_dir, exist_ok=True)
        plt.savefig(f"{save_dir}/metrics_comparison.png", dpi=300)
        plt.close()
    else:
        plt.show()

def save_results(forecasts: Dict[str, pd.Series], metrics: pd.DataFrame, save_dir: str):
    """Save forecasts and metrics to CSV files."""
    os.makedirs(save_dir, exist_ok=True)
    
    # Save forecasts
    valid_forecasts = {k: v for k, v in forecasts.items() if v is not None}
    if valid_forecasts:
        forecasts_df = pd.DataFrame(valid_forecasts)
        forecasts_df.to_csv(f"{save_dir}/forecasts.csv")
        logger.info(f"Forecasts saved to {save_dir}/forecasts.csv")
    else:
        logger.warning("No valid forecasts to save.")
    
    # Save metrics
    metrics.to_csv(f"{save_dir}/metrics.csv")
    logger.info(f"Metrics saved to {save_dir}/metrics.csv")
    
    # Save metrics as JSON for easy reading
    metrics_dict = metrics.to_dict(orient="index")
    with open(f"{save_dir}/metrics.json", "w") as f:
        json.dump(metrics_dict, f, indent=4)
    logger.info(f"Metrics saved to {save_dir}/metrics.json")

# ================ TRADITIONAL MODELS ================

def run_naive_forecast(train: pd.Series, test: pd.Series) -> Tuple[pd.Series, Dict[str, float]]:
    """Run naive forecast (last value)."""
    logger.info("Running Naive Forecast model...")
    forecast = pd.Series(train.iloc[-1], index=test.index)
    metrics = calculate_metrics(test, forecast, train)
    logger.info(f"Naive Forecast metrics: RMSE={metrics['RMSE']:.4f}, MAPE={metrics['MAPE']:.4f}")
    return forecast, metrics

def run_seasonal_naive_forecast(train: pd.Series, test: pd.Series, 
                              seasonal_period: int = 12) -> Tuple[pd.Series, Dict[str, float]]:
    """Run seasonal naive forecast (same value from last season)."""
    logger.info(f"Running Seasonal Naive Forecast model (period={seasonal_period})...")
    if len(train) < seasonal_period:
        logger.warning(f"Train data too short for seasonal naive with period {seasonal_period}. Using simple naive.")
        return run_naive_forecast(train, test)
    
    # Get the last seasonal cycle
    last_season = train.iloc[-seasonal_period:]
    
    # Create forecast by repeating the last season
    forecast_values = []
    for i in range(len(test)):
        idx = i % len(last_season)
        forecast_values.append(last_season.iloc[idx])
    
    forecast = pd.Series(forecast_values, index=test.index)
    metrics = calculate_metrics(test, forecast, train)
    logger.info(f"Seasonal Naive Forecast metrics: RMSE={metrics['RMSE']:.4f}, MAPE={metrics['MAPE']:.4f}")
    return forecast, metrics

def run_average_forecast(train: pd.Series, test: pd.Series, 
                       window: int = None) -> Tuple[pd.Series, Dict[str, float]]:
    """Run average forecast (mean of last n values or all values)."""
    if window and window < len(train):
        logger.info(f"Running Moving Average Forecast model (window={window})...")
        avg_value = train.iloc[-window:].mean()
        model_name = f"MovingAverage(window={window})"
    else:
        logger.info("Running Simple Average Forecast model...")
        avg_value = train.mean()
        model_name = "SimpleAverage"
    
    forecast = pd.Series(avg_value, index=test.index)
    metrics = calculate_metrics(test, forecast, train)
    logger.info(f"{model_name} metrics: RMSE={metrics['RMSE']:.4f}, MAPE={metrics['MAPE']:.4f}")
    return forecast, metrics

def run_drift_forecast(train: pd.Series, test: pd.Series) -> Tuple[pd.Series, Dict[str, float]]:
    """Run drift forecast (linear trend based on first and last value)."""
    logger.info("Running Drift Forecast model...")
    if len(train) < 2:
        logger.warning("Train data too short for drift model. Using naive forecast.")
        return run_naive_forecast(train, test)
    
    # Calculate slope
    y1, y2 = train.iloc[0], train.iloc[-1]
    t1, t2 = 0, len(train) - 1
    slope = (y2 - y1) / (t2 - t1)
    
    # Generate forecast
    forecast_values = []
    for i in range(len(test)):
        h = i + 1  # forecast horizon
        forecast_values.append(y2 + h * slope)
    
    forecast = pd.Series(forecast_values, index=test.index)
    metrics = calculate_metrics(test, forecast, train)
    logger.info(f"Drift Forecast metrics: RMSE={metrics['RMSE']:.4f}, MAPE={metrics['MAPE']:.4f}")
    return forecast, metrics

def run_ses_forecast(train: pd.Series, test: pd.Series) -> Tuple[pd.Series, Dict[str, float]]:
    """Run Simple Exponential Smoothing forecast."""
    logger.info("Running Simple Exponential Smoothing model...")
    try:
        model = ExponentialSmoothing(
            train, 
            trend=None, 
            seasonal=None,
            initialization_method="estimated"
        ).fit(optimized=True)
        
        forecast = model.forecast(steps=len(test))
        forecast = pd.Series(forecast.values, index=test.index)
        metrics = calculate_metrics(test, forecast, train)
        logger.info(f"SES Forecast metrics: RMSE={metrics['RMSE']:.4f}, MAPE={metrics['MAPE']:.4f}")
        return forecast, metrics
    except Exception as e:
        logger.error(f"Error in SES model: {e}")
        logger.info("Falling back to naive forecast.")
        return run_naive_forecast(train, test)

def run_holt_forecast(train: pd.Series, test: pd.Series) -> Tuple[pd.Series, Dict[str, float]]:
    """Run Holt's Exponential Smoothing (with trend) forecast."""
    logger.info("Running Holt\'s Exponential Smoothing model...")
    try:
        model = ExponentialSmoothing(
            train, 
            trend="add", 
            seasonal=None,
            initialization_method="estimated"
        ).fit(optimized=True)
        
        forecast = model.forecast(steps=len(test))
        forecast = pd.Series(forecast.values, index=test.index)
        metrics = calculate_metrics(test, forecast, train)
        logger.info(f"Holt Forecast metrics: RMSE={metrics['RMSE']:.4f}, MAPE={metrics['MAPE']:.4f}")
        return forecast, metrics
    except Exception as e:
        logger.error(f"Error in Holt model: {e}")
        logger.info("Falling back to SES forecast.")
        return run_ses_forecast(train, test)

def run_holt_winters_forecast(train: pd.Series, test: pd.Series, 
                            seasonal_period: int = 12) -> Tuple[pd.Series, Dict[str, float]]:
    """Run Holt-Winters' Exponential Smoothing (with trend and seasonality) forecast."""
    logger.info(f"Running Holt-Winters' Exponential Smoothing model (period={seasonal_period})...")
    if len(train) < 2 * seasonal_period:
        logger.warning(f"Train data too short for Holt-Winters with period {seasonal_period}. Using Holt's method.")
        return run_holt_forecast(train, test)
    
    try:
        model = ExponentialSmoothing(
            train, 
            trend="add", 
            seasonal="add",
            seasonal_periods=seasonal_period,
            initialization_method="estimated"
        ).fit(optimized=True)
        
        forecast = model.forecast(steps=len(test))
        forecast = pd.Series(forecast.values, index=test.index)
        metrics = calculate_metrics(test, forecast, train)
        logger.info(f"Holt-Winters Forecast metrics: RMSE={metrics['RMSE']:.4f}, MAPE={metrics['MAPE']:.4f}")
        return forecast, metrics
    except Exception as e:
        logger.error(f"Error in Holt-Winters model: {e}")
        logger.info("Falling back to Holt forecast.")
        return run_holt_forecast(train, test)

def run_ets_forecast(train: pd.Series, test: pd.Series, 
                   seasonal_period: int = 12) -> Tuple[pd.Series, Dict[str, float]]:
    """Run ETS (Error, Trend, Seasonal) model forecast."""
    logger.info(f"Running ETS model (period={seasonal_period})...")
    if len(train) < 2 * seasonal_period:
        logger.warning(f"Train data too short for ETS with period {seasonal_period}. Using Holt's method.")
        return run_holt_forecast(train, test)
    
    try:
        model = ETSModel(
            train,
            error="add",
            trend="add",
            seasonal="add",
            seasonal_periods=seasonal_period
        ).fit(disp=False)
        
        forecast = model.forecast(steps=len(test))
        forecast = pd.Series(forecast.values, index=test.index)
        metrics = calculate_metrics(test, forecast, train)
        logger.info(f"ETS Forecast metrics: RMSE={metrics['RMSE']:.4f}, MAPE={metrics['MAPE']:.4f}")
        return forecast, metrics
    except Exception as e:
        logger.error(f"Error in ETS model: {e}")
        logger.info("Falling back to Holt-Winters forecast.")
        return run_holt_winters_forecast(train, test, seasonal_period)

def run_theta_forecast(train: pd.Series, test: pd.Series, 
                     seasonal_period: int = 12) -> Tuple[pd.Series, Dict[str, float]]:
    """Run Theta model forecast."""
    logger.info(f"Running Theta model (period={seasonal_period})...")
    if len(train) < 2 * seasonal_period:
        logger.warning(f"Train data too short for Theta with period {seasonal_period}. Using SES method.")
        return run_ses_forecast(train, test)
    
    try:
        model = ThetaModel(
            train,
            period=seasonal_period,
            deseasonalize=True
        ).fit()
        
        forecast = model.forecast(steps=len(test))
        forecast = pd.Series(forecast.values, index=test.index)
        metrics = calculate_metrics(test, forecast, train)
        logger.info(f"Theta Forecast metrics: RMSE={metrics['RMSE']:.4f}, MAPE={metrics['MAPE']:.4f}")
        return forecast, metrics
    except Exception as e:
        logger.error(f"Error in Theta model: {e}")
        logger.info("Falling back to ETS forecast.")
        return run_ets_forecast(train, test, seasonal_period)

def run_arima_forecast(train: pd.Series, test: pd.Series) -> Tuple[pd.Series, Dict[str, float]]:
    """Run ARIMA model forecast."""
    logger.info("Running ARIMA model...")
    try:
        # Check stationarity and determine differencing order
        d = check_stationarity(train)
        logger.info(f"Using differencing order d={d} for ARIMA")
        
        # Use auto_arima-like approach to find best p, q
        best_aic = float("inf")
        best_order = (1, d, 1)  # Default
        
        max_p = min(5, len(train) // 10)
        max_q = min(5, len(train) // 10)
        
        for p in range(6):
            for q in range(6):
                if p == 0 and q == 0:
                    continue  # Skip ARIMA(0,d,0)
                
                try:
                    model = SARIMAX(
                        train,
                        order=(p, d, q),
                        simple_differencing=True
                    ).fit(disp=False)
                    
                    aic = model.aic
                    if aic < best_aic:
                        best_aic = aic
                        best_order = (p, d, q)
                except:
                    continue
        
        logger.info(f"Best ARIMA order: {best_order}")
        
        # Fit final model with best order
        final_model = SARIMAX(
            train,
            order=best_order,
            simple_differencing=True
        ).fit(disp=False)
        
        forecast = final_model.forecast(steps=len(test))
        forecast = pd.Series(forecast, index=test.index)
        metrics = calculate_metrics(test, forecast, train)
        logger.info(f"ARIMA{best_order} Forecast metrics: RMSE={metrics['RMSE']:.4f}, MAPE={metrics['MAPE']:.4f}")
        return forecast, metrics
    except Exception as e:
        logger.error(f"Error in ARIMA model: {e}")
        logger.info("Falling back to ETS forecast.")
        return run_ets_forecast(train, test)

from statsmodels.tsa.statespace.sarimax import SARIMAX
from typing import Tuple, Dict
import pandas as pd
import numpy as np

def run_sarima_forecast(train: pd.Series, test: pd.Series, 
                      seasonal_period: int = 12) -> Tuple[pd.Series, Dict[str, float]]:
    """Run improved SARIMA forecast using statsmodels only."""
    logger.info(f"Running improved SARIMA model (period={seasonal_period})...")

    if len(train) < 2 * seasonal_period:
        logger.warning(f"Train data too short for SARIMA with period {seasonal_period}. Using ARIMA.")
        return run_arima_forecast(train, test)

    try:
        # Determine d and D based on stationarity checks
        d = check_stationarity(train)
        D = 1 if train.diff(seasonal_period).dropna().autocorr() > 0.3 else 0
        
        logger.info(f"Using d={d}, D={D} for SARIMA")

        best_aic = float("inf")
        best_order = (0, d, 0)
        best_seasonal_order = (0, D, 0, seasonal_period)
        best_model = None

        # Expanded grid search ranges
        p_values = range(0, 3)
        q_values = range(0, 3)
        P_values = range(0, 2)
        Q_values = range(0, 2)

        for p in p_values:
            for q in q_values:
                for P in P_values:
                    for Q in Q_values:
                        try:
                            model = SARIMAX(
                                train,
                                order=(p, d, q),
                                seasonal_order=(P, D, Q, seasonal_period),
                                enforce_stationarity=False,
                                enforce_invertibility=False
                            ).fit(disp=False)
                            aic = model.aic
                            if aic < best_aic:
                                best_aic = aic
                                best_order = (p, d, q)
                                best_seasonal_order = (P, D, Q, seasonal_period)
                                best_model = model
                        except Exception as e:
                            logger.warning(f"SARIMA({p},{d},{q})x({P},{D},{Q},{seasonal_period}) failed: {e}")
                            continue
        
        logger.info(f"Best SARIMA order: {best_order} seasonal_order: {best_seasonal_order}, AIC={best_aic:.2f}")

        # Forecast
        forecast = best_model.forecast(steps=len(test))
        forecast = pd.Series(forecast, index=test.index)

        metrics = calculate_metrics(test, forecast, train)
        logger.info(f"Improved SARIMA Forecast metrics: RMSE={metrics['RMSE']:.4f}, MAPE={metrics['MAPE']:.4f}")
        return forecast, metrics

    except Exception as e:
        logger.error(f"Error in improved SARIMA model: {e}")
        logger.info("Falling back to ARIMA forecast.")
        return run_arima_forecast(train, test)


def run_prophet_forecast(train: pd.Series, test: pd.Series) -> Tuple[pd.Series, Dict[str, float]]:
    """Chạy mô hình Prophet với cấu hình mặc định."""
    logger.info("Chạy mô hình Prophet...")
    try:
        # Chuẩn bị dữ liệu
        df_prophet = pd.DataFrame({"ds": train.index, "y": train.values})
        
        # Khởi tạo Prophet
        model = Prophet(
            yearly_seasonality=True,
            weekly_seasonality=False,
            daily_seasonality=False,
            seasonality_mode="additive",
            changepoint_prior_scale=0.1
        )
        model.add_seasonality(name="monthly", period=30.5, fourier_order=10)
        model.fit(df_prophet)
        
        # Dự báo
        future = pd.DataFrame({"ds": test.index})
        forecast_df = model.predict(future)
        forecast = pd.Series(forecast_df["yhat"].values, index=test.index)
        
        # Tính chỉ số
        metrics = calculate_metrics(test, forecast, train)
        logger.info(f"Prophet Forecast metrics: RMSE={metrics['RMSE']:.4f}, MAPE={metrics['MAPE']:.4f}")
        return forecast, metrics
    
    except Exception as e:
        logger.error(f"Lỗi trong mô hình Prophet: {e}")
        logger.info("Quay về SARIMA forecast.")
        return run_sarima_forecast(train, test)


# ================ MACHINE LEARNING MODELS ================

def prepare_ml_features(df: pd.DataFrame, target_col: str = "cpi") -> Tuple[pd.DataFrame, pd.Series]:
    """Prepare features and target for machine learning models."""
    logger.info("Preparing features for ML models...")
    
    if df.empty or target_col not in df.columns:
        logger.error("Input DataFrame is empty or target column missing.")
        return pd.DataFrame(), pd.Series(dtype=float)
    
    # Create features
    features_df = create_features(df, target_col)
    
    # Extract target
    target = df[target_col]
    
    # Align features and target by index
    common_index = features_df.index.intersection(target.index)
    if len(common_index) == 0:
        logger.error("No common indices between features and target.")
        return pd.DataFrame(), pd.Series(dtype=float)
    
    features_df = features_df.loc[common_index]
    target = target.loc[common_index]
    
    logger.info(f"ML features prepared. X shape: {features_df.shape}, y shape: {target.shape}")
    return features_df, target

def run_ml_model(model_name: str, model_class, X_train: pd.DataFrame, y_train: pd.Series, 
                 X_test: pd.DataFrame, y_test: pd.Series, 
                 train_series: pd.Series, use_optuna: bool = True) -> Tuple[pd.Series, Dict[str, float]]:
    logger.info(f"Running {model_name} model...")
    
    # Kiểm tra căn chỉnh chỉ số
    if not X_train.index.equals(y_train.index) or not X_test.index.equals(y_test.index):
        logger.error(f"Index misalignment in {model_name}. Aligning data...")
        common_train_idx = X_train.index.intersection(y_train.index)
        common_test_idx = X_test.index.intersection(y_test.index)
        X_train = X_train.loc[common_train_idx]
        y_train = y_train.loc[common_train_idx]
        X_test = X_test.loc[common_test_idx]
        y_test = y_test.loc[common_test_idx]
    
    if X_train.empty or X_test.empty:
        logger.error(f"Empty feature matrices for {model_name}. Skipping.")
        return None, None
    
    try:
        # Scale features if needed
        if model_name in ["SVR", "ElasticNet", "KNN"]:
            scaler = StandardScaler()
            X_train_scaled = scaler.fit_transform(X_train)
            X_test_scaled = scaler.transform(X_test)
        else:
            X_train_scaled = X_train
            X_test_scaled = X_test
        
        # Hyperparameter optimization with Optuna
        if use_optuna and optuna_available:
            logger.info(f"Optimizing hyperparameters for {model_name} with Optuna...")
            
            def objective(trial):
                if model_name == "RandomForest":
                    params = {
                        "n_estimators": trial.suggest_int("n_estimators", 50, 300),
                        "max_depth": trial.suggest_int("max_depth", 5, 30),
                        "min_samples_split": trial.suggest_int("min_samples_split", 2, 20),
                        "min_samples_leaf": trial.suggest_int("min_samples_leaf", 1, 10),
                        "random_state": CONFIG['random_seed']
                    }
                elif model_name == "XGBoost":
                    params = {
                        "n_estimators": trial.suggest_int("n_estimators", 50, 300),
                        "max_depth": trial.suggest_int("max_depth", 3, 12),
                        "learning_rate": trial.suggest_float("learning_rate", 0.01, 0.3, log=True),
                        "subsample": trial.suggest_float("subsample", 0.6, 1.0),
                        "colsample_bytree": trial.suggest_float("colsample_bytree", 0.6, 1.0),
                        "random_state": CONFIG['random_seed']
                    }
                elif model_name == "LightGBM":
                    params = {
                        "n_estimators": trial.suggest_int("n_estimators", 50, 300),
                        "max_depth": trial.suggest_int("max_depth", 3, 12),
                        "learning_rate": trial.suggest_float("learning_rate", 0.01, 0.3, log=True),
                        "num_leaves": trial.suggest_int("num_leaves", 10, 100),
                        "subsample": trial.suggest_float("subsample", 0.6, 1.0),
                        "colsample_bytree": trial.suggest_float("colsample_bytree", 0.6, 1.0),
                        "random_state": CONFIG['random_seed']
                    }
                elif model_name == "SVR":
                    params = {
                        "C": trial.suggest_float("C", 0.1, 100.0, log=True),
                        "epsilon": trial.suggest_float("epsilon", 0.01, 1.0, log=True),
                        "gamma": trial.suggest_categorical("gamma", ["scale", "auto"])
                    }
                elif model_name == "GradientBoosting":
                    params = {
                        "n_estimators": trial.suggest_int("n_estimators", 50, 300),
                        "max_depth": trial.suggest_int("max_depth", 3, 12),
                        "learning_rate": trial.suggest_float("learning_rate", 0.01, 0.3, log=True),
                        "subsample": trial.suggest_float("subsample", 0.6, 1.0),
                        "random_state": CONFIG['random_seed']
                    }
                elif model_name == "ElasticNet":
                    params = {
                        "alpha": trial.suggest_float("alpha", 0.0001, 1.0, log=True),
                        "l1_ratio": trial.suggest_float("l1_ratio", 0.0, 1.0),
                        "random_state": CONFIG['random_seed']
                    }
                elif model_name == "KNN":
                    params = {
                        "n_neighbors": trial.suggest_int("n_neighbors", 3, 15),
                        "weights": trial.suggest_categorical("weights", ["uniform", "distance"]),
                        "p": trial.suggest_categorical("p", [1, 2])  # 1: Manhattan, 2: Euclidean
                    }
                else:
                    params = {}
                
                # Create validation split (using time series split)
                split_idx = int(len(X_train_scaled) * 0.8)
                X_t, X_v = X_train_scaled[:split_idx], X_train_scaled[split_idx:]
                y_t, y_v = y_train.iloc[:split_idx], y_train.iloc[split_idx:]
                
                # Train model
                model = model_class(**params)
                model.fit(X_t, y_t)
                
                # Evaluate
                preds = model.predict(X_v)
                return mean_squared_error(y_v, preds)
            
            # Run optimization
            study = optuna.create_study(direction="minimize")
            study.optimize(objective, n_trials=CONFIG['optuna_trials'])
            best_params = study.best_params
            logger.info(f"Best {model_name} params: {best_params}")
            
            # Train final model with best params
            model = model_class(**best_params)
        else:
            # Use default parameters
            logger.info(f"Using default parameters for {model_name}")
            if model_name == "RandomForest":
                model = RandomForestRegressor(n_estimators=100, random_state=CONFIG['random_seed'])
            elif model_name == "XGBoost":
                model = xgb.XGBRegressor(n_estimators=100, random_state=CONFIG['random_seed'])
            elif model_name == "LightGBM":
                model = lgb.LGBMRegressor(n_estimators=100, random_state=CONFIG['random_seed'])
            elif model_name == "SVR":
                model = SVR(C=1.0, epsilon=0.1, gamma="scale")
            elif model_name == "GradientBoosting":
                model = GradientBoostingRegressor(n_estimators=100, random_state=CONFIG['random_seed'])
            elif model_name == "ElasticNet":
                model = ElasticNet(alpha=0.1, l1_ratio=0.5, random_state=CONFIG['random_seed'])
            elif model_name == "KNN":
                model = KNeighborsRegressor(n_neighbors=5, weights="uniform")
            else:
                model = model_class()
        
        # Train final model
        model.fit(X_train_scaled, y_train)
        
        # Generate forecast
        forecast = model.predict(X_test_scaled)
        
        # Ensure forecast aligns with y_test index
        forecast = pd.Series(forecast, index=y_test.index)
        
        # Calculate metrics
        metrics = calculate_metrics(y_test, forecast, train_series)
        logger.info(f"{model_name} Forecast metrics: RMSE={metrics['RMSE']:.4f}, MAPE={metrics['MAPE']:.4f}")
        
        return forecast, metrics
    except Exception as e:
        logger.error(f"Error in {model_name} model: {e}")
        return None, None

# ================ DEEP LEARNING MODELS ================

class TimeSeriesDataset(TensorDataset):
    def __init__(self, X, y, seq_length):
        self.X = torch.FloatTensor(X)
        self.y = torch.FloatTensor(y)
        self.seq_length = seq_length
        
    def __len__(self):
        return len(self.X) - self.seq_length
    
    def __getitem__(self, idx):
        return (self.X[idx:idx+self.seq_length], self.y[idx+self.seq_length])

class Attention(nn.Module):
    """Cơ chế Attention đơn giản."""
    def __init__(self, hidden_size, bidirectional=False):
        super(Attention, self).__init__()
        self.bidirectional = bidirectional
        input_dim = hidden_size * 2 if bidirectional else hidden_size
        self.attention = nn.Linear(input_dim, 1)
    
    def forward(self, lstm_output):
        # lstm_output: [batch, seq_len, hidden_size] or [batch, seq_len, hidden_size * 2]
        attention_scores = self.attention(lstm_output).squeeze(-1)  # [batch, seq_len]
        attention_weights = torch.softmax(attention_scores, dim=1)  # [batch, seq_len]
        context_vector = torch.bmm(attention_weights.unsqueeze(1), lstm_output).squeeze(1)  # [batch, hidden_size]
        return context_vector

class LSTMWithAttention(nn.Module):
    def __init__(self, input_size, hidden_size, num_layers, output_size, dropout=0.2):
        super().__init__()
        self.lstm = nn.LSTM(
            input_size, hidden_size, num_layers, 
            batch_first=True, dropout=dropout if num_layers > 1 else 0
        )
        self.layer_norm = nn.LayerNorm(hidden_size)
        self.attention = Attention(hidden_size, bidirectional=False)
        self.fc = nn.Linear(hidden_size, output_size)
    
    def forward(self, x):
        lstm_out, _ = self.lstm(x)
        lstm_out = self.layer_norm(lstm_out)
        context = self.attention(lstm_out)
        out = self.fc(context)
        return out

class GRUWithAttention(nn.Module):
    def __init__(self, input_size, hidden_size, num_layers, output_size, dropout=0.2):
        super().__init__()
        self.gru = nn.GRU(
            input_size, hidden_size, num_layers, 
            batch_first=True, dropout=dropout if num_layers > 1 else 0
        )
        self.layer_norm = nn.LayerNorm(hidden_size)
        self.attention = Attention(hidden_size, bidirectional=False)
        self.fc = nn.Linear(hidden_size, output_size)
    
    def forward(self, x):
        gru_out, _ = self.gru(x)
        gru_out = self.layer_norm(gru_out)
        context = self.attention(gru_out)
        out = self.fc(context)
        return out

class BiLSTMWithAttention(nn.Module):
    def __init__(self, input_size, hidden_size, num_layers, output_size, dropout=0.2):
        super().__init__()
        self.lstm = nn.LSTM(
            input_size, hidden_size, num_layers, 
            batch_first=True, dropout=dropout if num_layers > 1 else 0,
            bidirectional=True
        )
        self.layer_norm = nn.LayerNorm(hidden_size * 2)  # *2 for bidirectional
        self.attention = Attention(hidden_size, bidirectional=True)
        self.fc = nn.Linear(hidden_size * 2, output_size)
    
    def forward(self, x):
        lstm_out, _ = self.lstm(x)
        lstm_out = self.layer_norm(lstm_out)
        context = self.attention(lstm_out)
        out = self.fc(context)
        return out

class CNNLSTMWithAttention(nn.Module):
    def __init__(self, input_size, hidden_size, num_layers, output_size, dropout=0.2):
        super().__init__()
        self.conv1 = nn.Conv1d(input_size, 64, kernel_size=3, padding=1)
        self.conv2 = nn.Conv1d(64, 32, kernel_size=3, padding=1)
        self.relu = nn.ReLU()
        self.pool = nn.MaxPool1d(kernel_size=2)
        self.lstm = nn.LSTM(
            32, hidden_size, num_layers, 
            batch_first=True, dropout=dropout if num_layers > 1 else 0
        )
        self.layer_norm = nn.LayerNorm(hidden_size)
        self.attention = Attention(hidden_size, bidirectional=False)
        self.fc = nn.Linear(hidden_size, output_size)
    
    def forward(self, x):
        x = x.transpose(1, 2)  # [batch, features, seq_len]
        x = self.relu(self.conv1(x))
        x = self.pool(self.relu(self.conv2(x)))  # [batch, 32, seq_len//2]
        x = x.transpose(1, 2)  # [batch, seq_len//2, 32]
        lstm_out, _ = self.lstm(x)
        lstm_out = self.layer_norm(lstm_out)
        context = self.attention(lstm_out)
        out = self.fc(context)
        return out
    
class TransformerModel(nn.Module):
    def __init__(self, input_size, d_model, nhead, num_layers, dropout=0.2):
        super().__init__()
        self.input_fc = nn.Linear(input_size, d_model)
        self.transformer = nn.TransformerEncoder(
            nn.TransformerEncoderLayer(
                d_model=d_model,
                nhead=nhead,
                dim_feedforward=d_model * 4,
                dropout=dropout,
                batch_first=True
            ),
            num_layers=num_layers
        )
        self.fc = nn.Linear(d_model, 1)
        self.relu = nn.ReLU()
    
    def forward(self, x):
        x = self.relu(self.input_fc(x))
        x = self.transformer(x)
        x = self.fc(x[:, -1, :])  # Lấy output cuối cùng của chuỗi
        return x
    
def create_sequences(X: np.ndarray, y: np.ndarray, seq_length: int) -> Tuple[np.ndarray, np.ndarray]:
    """Create sequences for deep learning models."""
    logger.info(f"Creating sequences with length {seq_length}...")
    Xs, ys = [], []
    for i in range(len(X) - seq_length):
        Xs.append(X[i:(i + seq_length)])
        ys.append(y[i + seq_length])
    
    if not Xs:
        logger.warning(f"No sequences created. Input length {len(X)} too short for seq_length {seq_length}.")
        return np.array([]), np.array([])
    
    Xs, ys = np.array(Xs), np.array(ys)
    logger.info(f"Sequences created. X shape: {Xs.shape}, y shape: {ys.shape}")
    return Xs, ys

from sklearn.preprocessing import RobustScaler
from scipy.stats import zscore

def prepare_dl_data(X_train: pd.DataFrame, y_train: pd.Series, X_test: pd.DataFrame, y_test: pd.Series, seq_length: int) -> Tuple[np.ndarray, np.ndarray, np.ndarray, Optional[RobustScaler], Optional[RobustScaler]]:
    """Chuẩn bị dữ liệu cho các mô hình học sâu."""
    logger.info("Preparing data for deep learning models with RobustScaler...")
    
    if X_train.empty or y_train.empty:
        logger.error("Training features or target is empty.")
        return np.array([]), np.array([]), np.array([]), None, None
    
    if len(X_train) < seq_length:
        logger.error(f"Training data too short ({len(X_train)}) for sequence length {seq_length}.")
        return np.array([]), np.array([]), np.array([]), None, None
    
    # Outlier detection
    y_train_clean = y_train[np.abs(zscore(y_train)) < 3]
    X_train_clean = X_train.loc[y_train_clean.index]
    
    # Handle empty or insufficient test data
    if X_test.empty or len(X_test) < 1:
        logger.warning("Test features empty. Using last training data for prediction.")
        X_test = X_train_clean.iloc[-seq_length:].copy()
        y_test = y_train_clean.iloc[-seq_length:].copy()
    
    # Ensure consistent columns
    missing_cols = set(X_train_clean.columns) - set(X_test.columns)
    for col in missing_cols:
        X_test[col] = 0
    X_test = X_test[X_train_clean.columns]
    
    # Scale data
    y_scaler = RobustScaler()
    X_scaler = RobustScaler()
    
    try:
        y_train_scaled = y_scaler.fit_transform(y_train_clean.values.reshape(-1, 1)).flatten()
        X_train_scaled = X_scaler.fit_transform(X_train_clean)
        X_test_scaled = X_scaler.transform(X_test)
    except ValueError as e:
        logger.error(f"Error scaling data: {e}")
        return np.array([]), np.array([]), np.array([]), None, None
    
    # Create sequences
    X_train_seq, y_train_seq = create_sequences(X_train_scaled, y_train_scaled, seq_length)
    
    if X_train_seq.size == 0:
        logger.error("Failed to create training sequences.")
        return np.array([]), np.array([]), np.array([]), None, None
    
    # Prepare prediction input
    X_pred_input = X_test_scaled
    if len(X_train_scaled) >= seq_length:
        last_train_seq = X_train_scaled[-seq_length:]
        if len(X_test_scaled) > 0:
            X_pred_input = np.concatenate([last_train_seq[:-1], X_test_scaled], axis=0)
        else:
            X_pred_input = last_train_seq
    
    logger.info(f"DL data prepared. X_train_seq shape: {X_train_seq.shape}, y_train_seq shape: {y_train_seq.shape}, X_pred_input shape: {X_pred_input.shape}")
    return X_train_seq, y_train_seq, X_pred_input, y_scaler, X_scaler

import torch
from torch.utils.data import DataLoader, TensorDataset
import optuna
import numpy as np
from sklearn.metrics import mean_squared_error

from torch.cuda.amp import autocast, GradScaler
import optuna

def run_dl_model(model_name: str, X_train_seq, y_train_seq, X_pred_input, 
                 y_test: pd.Series, y_scaler, seq_length, train_series,
                 use_optuna: bool = True) -> Tuple[pd.Series, Dict[str, float]]:
    """Chạy dự báo mô hình học sâu với Attention, LayerNorm, Mixed Precision, và Optuna."""
    logger.info(f"Running {model_name} with enhanced features...")
    
    if len(X_train_seq) == 0:
        logger.error(f"No sufficient data to create sequences for {model_name}.")
        return None, None
    
    try:
        # Tối ưu hóa siêu tham số với Optuna
        if use_optuna and optuna_available:
            logger.info(f"Optimizing hyperparameters for {model_name} with Optuna...")
            
            def objective(trial):
                hidden_size = trial.suggest_int("hidden_size", 16, 512)
                num_layers = trial.suggest_int("num_layers", 1, 6)
                dropout = trial.suggest_float("dropout", 0.0, 0.5)
                learning_rate = trial.suggest_float("learning_rate", 1e-5, 1e-2, log=True)
                batch_size = trial.suggest_categorical("batch_size", [16, 32, 64, 128, 256])
                optimizer_name = trial.suggest_categorical("optimizer", ["Adam", "AdamW", "RMSprop"])
                
                dataset = TensorDataset(
                    torch.FloatTensor(X_train_seq), 
                    torch.FloatTensor(y_train_seq).unsqueeze(1)
                )
                train_size = int(0.8 * len(dataset))
                val_size = len(dataset) - train_size
                train_dataset, val_dataset = torch.utils.data.random_split(dataset, [train_size, val_size])
                
                train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
                val_loader = DataLoader(val_dataset, batch_size=batch_size)
                
                device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
                input_size = X_train_seq.shape[2]
                
                if model_name == "LSTM":
                    model = LSTMWithAttention(input_size, hidden_size, num_layers, 1, dropout).to(device)
                elif model_name == "GRU":
                    model = GRUWithAttention(input_size, hidden_size, num_layers, 1, dropout).to(device)
                elif model_name == "BiLSTM":
                    model = BiLSTMWithAttention(input_size, hidden_size, num_layers, 1, dropout).to(device)
                elif model_name == "CNN-LSTM":
                    model = CNNLSTMWithAttention(input_size, hidden_size, num_layers, 1, dropout).to(device)
                
                # Sử dụng DataParallel nếu có nhiều GPU
                if torch.cuda.device_count() > 1:
                    model = nn.DataParallel(model)
                
                criterion = nn.MSELoss()
                if optimizer_name == "Adam":
                    optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate, weight_decay=1e-5)
                elif optimizer_name == "AdamW":
                    optimizer = torch.optim.AdamW(model.parameters(), lr=learning_rate, weight_decay=1e-5)
                else:
                    optimizer = torch.optim.RMSprop(model.parameters(), lr=learning_rate, weight_decay=1e-5)
                
                scaler = GradScaler()  # Mixed Precision
                model.train()
                for epoch in range(20):  # Giảm epoch cho Optuna
                    for X_batch, y_batch in train_loader:
                        X_batch, y_batch = X_batch.to(device), y_batch.to(device)
                        optimizer.zero_grad()
                        with autocast():
                            y_pred = model(X_batch)
                            loss = criterion(y_pred, y_batch)
                        scaler.scale(loss).backward()
                        scaler.step(optimizer)
                        scaler.update()
                
                model.eval()
                val_loss = 0
                with torch.no_grad():
                    for X_batch, y_batch in val_loader:
                        X_batch, y_batch = X_batch.to(device), y_batch.to(device)
                        with autocast():
                            y_pred = model(X_batch)
                            val_loss += criterion(y_pred, y_batch).item()
                
                return val_loss / len(val_loader)
            
            study = optuna.create_study(direction="minimize", pruner=optuna.pruners.MedianPruner(n_warmup_steps=5))
            study.optimize(objective, n_trials=200)  # Tăng n_trials
            best_params = study.best_params
            logger.info(f"Best {model_name} params: {best_params}")
            
            hidden_size = best_params["hidden_size"]
            num_layers = best_params["num_layers"]
            dropout = best_params["dropout"]
            learning_rate = best_params["learning_rate"]
            batch_size = best_params["batch_size"]
            optimizer_name = best_params["optimizer"]
        else:
            hidden_size = 64
            num_layers = 2
            dropout = 0.2
            learning_rate = 0.001
            batch_size = 32
            optimizer_name = "Adam"
        
        # Tạo dataset
        dataset = TensorDataset(
            torch.FloatTensor(X_train_seq), 
            torch.FloatTensor(y_train_seq).unsqueeze(1)
        )
        train_loader = DataLoader(dataset, batch_size=batch_size, shuffle=True)
        
        # Tạo mô hình
        device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
        input_size = X_train_seq.shape[2]
        
        if model_name == "LSTM":
            model = LSTMWithAttention(input_size, hidden_size, num_layers, 1, dropout).to(device)
        elif model_name == "GRU":
            model = GRUWithAttention(input_size, hidden_size, num_layers, 1, dropout).to(device)
        elif model_name == "BiLSTM":
            model = BiLSTMWithAttention(input_size, hidden_size, num_layers, 1, dropout).to(device)
        elif model_name == "CNN-LSTM":
            model = CNNLSTMWithAttention(input_size, hidden_size, num_layers, 1, dropout).to(device)
        
        if torch.cuda.device_count() > 1:
            model = nn.DataParallel(model)
        
        # Khởi tạo optimizer
        if optimizer_name == "Adam":
            optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate, weight_decay=1e-5)
        elif optimizer_name == "AdamW":
            optimizer = torch.optim.AdamW(model.parameters(), lr=learning_rate, weight_decay=1e-5)
        else:
            optimizer = torch.optim.RMSprop(model.parameters(), lr=learning_rate, weight_decay=1e-5)
        
        criterion = nn.MSELoss()
        scaler = GradScaler()
        scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer, mode="min", factor=0.5, patience=5)
        
        # Huấn luyện với Mixed Precision và Checkpointing
        model.train()
        best_loss = float("inf")
        patience_counter = 0
        checkpoint_path = f"{CONFIG['results_dir']}/{model_name}_best.pth"
        
        for epoch in range(CONFIG['dl_epochs']):
            epoch_loss = 0
            for X_batch, y_batch in train_loader:
                X_batch, y_batch = X_batch.to(device), y_batch.to(device)
                optimizer.zero_grad()
                with autocast():
                    y_pred = model(X_batch)
                    loss = criterion(y_pred, y_batch)
                scaler.scale(loss).backward()
                scaler.step(optimizer)
                scaler.update()
                epoch_loss += loss.item()
            
            avg_loss = epoch_loss / len(train_loader)
            scheduler.step(avg_loss)
            
            if avg_loss < best_loss:
                best_loss = avg_loss
                torch.save(model.state_dict(), checkpoint_path)
                patience_counter = 0
            else:
                patience_counter += 1
                if patience_counter >= CONFIG['dl_patience']:
                    logger.info(f"Early stopping at epoch {epoch+1}")
                    break
            
            if (epoch + 1) % 10 == 0 or epoch == 0:
                logger.info(f"Epoch {epoch+1}/{CONFIG['dl_epochs']}, Loss: {avg_loss:.6f}")
        
        # Tải mô hình tốt nhất
        model.load_state_dict(torch.load(checkpoint_path))
        
        # Dự báo
        model.eval()
        forecast_scaled = []
        with torch.no_grad():
            for i in range(len(y_test)):
                input_seq = X_pred_input[i:i + seq_length]
                if len(input_seq) < seq_length:
                    logger.warning(f"Input sequence too short at step {i}. Skipping.")
                    continue
                input_tensor = torch.FloatTensor(input_seq).unsqueeze(0).to(device)
                with autocast():
                    pred = model(input_tensor)
                forecast_scaled.append(pred.item())
        
        # Chuyển đổi ngược dự báo
        forecast = y_scaler.inverse_transform(np.array(forecast_scaled).reshape(-1, 1)).flatten()
        forecast = pd.Series(forecast, index=y_test.index[:len(forecast)])
        
        # Tính chỉ số
        metrics = calculate_metrics(y_test[:len(forecast)], forecast, train_series)
        logger.info(f"{model_name} Forecast metrics: RMSE={metrics['RMSE']:.4f}, MAPE={metrics['MAPE']:.4f}")
        
        return forecast, metrics
    except Exception as e:
        logger.error(f"Error in {model_name} model: {e}")
        return None, None
    
# ================ HYBRID AND ENSEMBLE MODELS ================
import optuna
from sklearn.metrics import mean_squared_error

def optimize_hyperparameters(X_train_seq, y_train_seq, X_test_seq, y_test, model_type="LSTM", target_scaler=None):
    """
    Tối ưu hóa siêu tham số với Optuna.
    
    Args:
        X_train_seq, X_test_seq: Chuỗi đặc trưng.
        y_train_seq, y_test: Mục tiêu.
        model_type: Loại mô hình ("LSTM", "GRU", "BiLSTM", "CNN-LSTM").
        target_scaler: Bộ chuẩn hóa mục tiêu.
    
    Returns:
        best_params: Siêu tham số tốt nhất.
    """
    def objective(trial):
        hidden_size = trial.suggest_int("hidden_size", 50, 200)
        num_layers = trial.suggest_int("num_layers", 1, 4)
        dropout = trial.suggest_float("dropout", 0.0, 0.5)
        learning_rate = trial.suggest_float("learning_rate", 1e-4, 1e-2, log=True)
        batch_size = trial.suggest_categorical("batch_size", [16, 32, 64])

        # Khởi tạo mô hình
        if model_type == "LSTM":
            model = LSTMWithAttention(input_size=X_train_seq.shape[2], hidden_size=hidden_size, 
                              num_layers=num_layers, dropout=dropout)
        elif model_type == "CNN-LSTM":
            model = CNNLSTMWithAttention(input_size=X_train_seq.shape[2], hidden_size=hidden_size, 
                                 num_layers=num_layers, dropout=dropout)
        # Thêm GRU, BiLSTM tương tự nếu cần

        optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate, weight_decay=1e-5)
        criterion = nn.MSELoss()

        # Huấn luyện
        model.train()
        X_train_seq_torch = torch.tensor(X_train_seq, dtype=torch.float32)
        y_train_seq_torch = torch.tensor(y_train_seq, dtype=torch.float32).unsqueeze(-1)
        for epoch in range(50):  # Giảm số epoch để tăng tốc Optuna
            for i in range(0, len(X_train_seq), batch_size):
                batch_X = X_train_seq_torch[i:i+batch_size]
                batch_y = y_train_seq_torch[i:i+batch_size]
                optimizer.zero_grad()
                outputs = model(batch_X)
                loss = criterion(outputs, batch_y)
                loss.backward()
                optimizer.step()

        # Đánh giá
        model.eval()
        with torch.no_grad():
            X_test_seq_torch = torch.tensor(X_test_seq, dtype=torch.float32)
            pred = model(X_test_seq_torch).numpy().flatten()
            if target_scaler:
                pred = target_scaler.inverse_transform(pred.reshape(-1, 1)).flatten()
            rmse = np.sqrt(mean_squared_error(y_test, pred))
        return rmse

    study = optuna.create_study(direction="minimize")
    study.optimize(objective, n_trials=100)  # Tăng số lần thử
    return study.best_params

def run_simple_average_ensemble(forecasts: Dict[str, pd.Series], train: pd.Series, test: pd.Series) -> Tuple[pd.Series, Dict[str, float]]:
    """Run simple average ensemble of multiple forecasts."""
    logger.info("Running Simple Average Ensemble...")
    
    # Filter out None forecasts
    valid_forecasts = {k: v for k, v in forecasts.items() if v is not None}
    if len(valid_forecasts) < 2:
        logger.error("Need at least 2 valid forecasts for ensemble. Skipping.")
        return None, None
    
    # Align forecasts to test index
    aligned_forecasts = {}
    for name, forecast in valid_forecasts.items():
        aligned_forecasts[name] = forecast.reindex(test.index)
    
    # Calculate ensemble forecast
    forecast_df = pd.DataFrame(aligned_forecasts)
    ensemble_forecast = forecast_df.mean(axis=1)
    
    # Calculate metrics
    metrics = calculate_metrics(test, ensemble_forecast, train)
    logger.info(f"Simple Average Ensemble metrics: RMSE={metrics['RMSE']:.4f}, MAPE={metrics['MAPE']:.4f}")
    
    return ensemble_forecast, metrics

def run_weighted_average_ensemble(forecasts: Dict[str, pd.Series], metrics_dict: Dict[str, Dict[str, float]], 
                                train: pd.Series, test: pd.Series, 
                                weight_metric: str = "RMSE") -> Tuple[pd.Series, Dict[str, float]]:
    """Run weighted average ensemble based on model performance."""
    logger.info(f"Running Weighted Average Ensemble (weight_metric={weight_metric})...")
    
    # Filter out None forecasts and ensure metrics exist
    valid_forecasts = {}
    weights = {}
    
    for name, forecast in forecasts.items():
        if forecast is not None and name in metrics_dict and metrics_dict[name] is not None:
            valid_forecasts[name] = forecast.reindex(test.index)
            metric_value = metrics_dict[name].get(weight_metric)
            if metric_value is not None and not np.isnan(metric_value) and metric_value > 1e-9:
                # Lower metric is better, so use inverse
                weights[name] = 1.0 / metric_value
            elif metric_value is not None and not np.isnan(metric_value):
                 weights[name] = 1e9 # Assign large weight if metric is near zero
    
    if len(valid_forecasts) < 2 or not weights:
        logger.error("Need at least 2 valid forecasts with metrics for weighted ensemble. Skipping.")
        return None, None
    
    # Normalize weights
    total_weight = sum(weights.values())
    if total_weight < 1e-9:
        logger.error("Total weight is zero. Cannot normalize. Skipping weighted ensemble.")
        return None, None
        
    normalized_weights = {k: w / total_weight for k, w in weights.items()}
    
    logger.info(f"Ensemble weights: {normalized_weights}")
    
    # Calculate weighted ensemble forecast
    forecast_df = pd.DataFrame(valid_forecasts)
    ensemble_forecast = pd.Series(0.0, index=test.index)
    
    for name, weight in normalized_weights.items():
        ensemble_forecast += forecast_df[name] * weight
    
    # Calculate metrics
    metrics = calculate_metrics(test, ensemble_forecast, train)
    logger.info(f"Weighted Average Ensemble metrics: RMSE={metrics['RMSE']:.4f}, MAPE={metrics['MAPE']:.4f}")
    
    return ensemble_forecast, metrics

from sklearn.linear_model import RidgeCV
from sklearn.impute import SimpleImputer

def run_stacking_ensemble(forecasts: Dict[str, pd.Series], train_forecasts: Dict[str, pd.Series],
                         train: pd.Series, test: pd.Series) -> Tuple[pd.Series, Dict[str, float]]:
    logger.info("Running Stacking Ensemble...")
    
    valid_forecasts = {k: v for k, v in forecasts.items() if v is not None}
    valid_train_forecasts = {k: v for k, v in train_forecasts.items() if v is not None}
    
    common_models = sorted(list(set(valid_forecasts.keys()).intersection(set(valid_train_forecasts.keys()))))
    if len(common_models) < 2:
        logger.error(f"Need at least 2 common models. Got {len(common_models)}. Skipping.")
        return None, None
    
    X_meta_train_dict = {model: valid_train_forecasts[model] for model in common_models}
    X_meta_train = pd.DataFrame(X_meta_train_dict)
    y_meta_train = train
    
    common_idx_train = X_meta_train.index.intersection(y_meta_train.index)
    if len(common_idx_train) == 0:
        logger.error("No common indices between train forecasts and target. Skipping.")
        return None, None
    
    X_meta_train = X_meta_train.loc[common_idx_train]
    y_meta_train = y_meta_train.loc[common_idx_train]
    
    if X_meta_train.isna().any().any() or y_meta_train.isna().any():
        logger.warning("NaN values detected in meta-training data. Applying imputation...")
        imputer = SimpleImputer(strategy='mean')
        X_meta_train = pd.DataFrame(imputer.fit_transform(X_meta_train), 
                                   columns=X_meta_train.columns, index=X_meta_train.index)
        y_meta_train = y_meta_train.fillna(y_meta_train.mean())
    
    if X_meta_train.empty or y_meta_train.empty:
        logger.error("Meta-training data empty after alignment. Skipping.")
        return None, None
    
    X_meta_test_dict = {model: valid_forecasts[model].reindex(test.index) for model in common_models}
    X_meta_test = pd.DataFrame(X_meta_test_dict)
    
    if X_meta_test.isna().any().any():
        logger.warning("NaN values detected in meta-test data. Applying imputation...")
        X_meta_test = pd.DataFrame(imputer.transform(X_meta_test), 
                                  columns=X_meta_test.columns, index=X_meta_test.index)
    
    try:
        meta_model = RidgeCV(alphas=[0.01, 0.1, 1.0, 10.0, 100.0])
        meta_model.fit(X_meta_train, y_meta_train)
    except ValueError as e:
        logger.error(f"Error training meta-model: {e}")
        return None, None
    
    ensemble_forecast = meta_model.predict(X_meta_test)
    ensemble_forecast = pd.Series(ensemble_forecast, index=test.index)
    
    metrics = calculate_metrics(test, ensemble_forecast, train)
    logger.info(f"Stacking Ensemble metrics: RMSE={metrics['RMSE']:.4f}, MAPE={metrics['MAPE']:.4f}")
    logger.info(f"Meta-model coefficients: {dict(zip(common_models, meta_model.coef_))}")
    
    return ensemble_forecast, metrics

def run_dynamic_ensemble(forecasts: Dict[str, pd.Series], train_forecasts: Dict[str, pd.Series],
                        train: pd.Series, test: pd.Series) -> Tuple[pd.Series, Dict[str, float]]:
    logger.info("Running Dynamic Ensemble...")
    
    try:
        valid_forecasts = {k: v for k, v in forecasts.items() if v is not None and not v.isna().all()}
        valid_train_forecasts = {k: v for k, v in train_forecasts.items() if v is not None and not v.isna().all()}
        common_models = sorted(list(set(valid_forecasts.keys()).intersection(set(valid_train_forecasts.keys()))))
        
        if len(common_models) < 2:
            logger.error("Need at least 2 common models for dynamic ensemble. Skipping.")
            return None, None
        
        model_scores = {}
        for model in common_models:
            train_pred = valid_train_forecasts[model].reindex(train.index)
            if train_pred.isna().all() or train_pred.empty:
                logger.warning(f"Skipping model {model} due to all NaN or empty train predictions.")
                continue
            rmse = np.sqrt(mean_squared_error(train, train_pred))
            model_scores[model] = rmse
        
        top_models = sorted(model_scores, key=model_scores.get)[:min(5, len(model_scores))]
        if not top_models:
            logger.error("No valid models for dynamic ensemble. Skipping.")
            return None, None
        
        logger.info(f"Selected top models for dynamic ensemble: {top_models}")
        
        X_meta_train = pd.DataFrame({model: valid_train_forecasts[model] for model in top_models}).reindex(train.index)
        X_meta_test = pd.DataFrame({model: valid_forecasts[model] for model in top_models}).reindex(test.index)
        y_meta_train = train
        
        nan_columns_train = X_meta_train.columns[X_meta_train.isna().all()]
        nan_columns_test = X_meta_test.columns[X_meta_test.isna().all()]
        if nan_columns_train.any() or nan_columns_test.any():
            logger.warning(f"Removing all-NaN columns: {nan_columns_train.tolist()} (train), {nan_columns_test.tolist()} (test)")
            X_meta_train = X_meta_train.drop(columns=nan_columns_train, errors='ignore')
            X_meta_test = X_meta_test.drop(columns=nan_columns_test, errors='ignore')
            top_models = [m for m in top_models if m not in nan_columns_train and m not in nan_columns_test]
        
        if len(top_models) < 2:
            logger.error("Not enough valid models after removing NaN columns. Skipping.")
            return None, None
        
        imputer = SimpleImputer(strategy='mean')
        X_meta_train = pd.DataFrame(imputer.fit_transform(X_meta_train), 
                                   columns=X_meta_train.columns, index=X_meta_train.index)
        X_meta_test = pd.DataFrame(imputer.transform(X_meta_test), 
                                  columns=X_meta_test.columns, index=X_meta_test.index)
        
        if X_meta_train.isna().any().any() or X_meta_test.isna().any().any():
            logger.error("NaN values remain after imputation.")
            return None, None
        if np.isinf(X_meta_train).any().any() or np.isinf(X_meta_test).any().any():
            logger.error("Infinite values in meta-data.")
            return None, None
        
        # Optimize LightGBM hyperparameters
        if optuna_available:
            def objective(trial):
                params = {
                    "n_estimators": trial.suggest_int("n_estimators", 50, 300),
                    "max_depth": trial.suggest_int("max_depth", 3, 12),
                    "learning_rate": trial.suggest_float("learning_rate", 0.01, 0.3, log=True),
                    "subsample": trial.suggest_float("subsample", 0.6, 1.0),
                    "colsample_bytree": trial.suggest_float("colsample_bytree", 0.6, 1.0),
                    "random_state": CONFIG['random_seed']
                }
                model = lgb.LGBMRegressor(**params)
                model.fit(X_meta_train, y_meta_train)
                pred = model.predict(X_meta_test)
                return mean_squared_error(test, pred)
            
            study = optuna.create_study(direction="minimize")
            study.optimize(objective, n_trials=CONFIG['optuna_trials'])
            best_params = study.best_params
            logger.info(f"Best LightGBM params for dynamic ensemble: {best_params}")
        else:
            best_params = {
                "n_estimators": 100,
                "max_depth": 5,
                "learning_rate": 0.1,
                "subsample": 0.8,
                "colsample_bytree": 0.8,
                "random_state": CONFIG['random_seed']
            }
        
        lgb_model = lgb.LGBMRegressor(**best_params)
        lgb_model.fit(X_meta_train, y_meta_train)
        
        ensemble_forecast = lgb_model.predict(X_meta_test)
        ensemble_forecast = pd.Series(ensemble_forecast, index=test.index)
        
        metrics = calculate_metrics(test, ensemble_forecast, train)
        logger.info(f"Dynamic Ensemble metrics: RMSE={metrics['RMSE']:.4f}, MAPE={metrics['MAPE']:.4f}")
        
        return ensemble_forecast, metrics
    except Exception as e:
        logger.error(f"Error in Dynamic Ensemble: {e}")
        return None, None
    

def run_hybrid_sarima_bilstm(train: pd.Series, test: pd.Series, 
                             X_train: pd.DataFrame, y_train: pd.Series, 
                             X_test: pd.DataFrame, y_test: pd.Series) -> Tuple[pd.Series, Dict[str, float]]:
    """Run an enhanced Hybrid SARIMA-BiLSTM model with dynamic SARIMA order selection."""
    logger.info("Running Enhanced Hybrid SARIMA-BiLSTM...")
    if not torch_available:
        logger.warning("PyTorch not available. Skipping Hybrid SARIMA-BiLSTM.")
        return None, None
    
    try:
        # Step 1: Dynamic SARIMA order selection
        logger.info("Performing dynamic SARIMA order selection...")
        d = check_stationarity(train)
        D = 1 if train.diff(12).dropna().autocorr() > 0.3 else 0
        best_aic = float("inf")
        best_order = (1, d, 1)
        best_seasonal_order = (1, D, 1, 12)
        
        for p in range(3):
            for q in range(3):
                for P in range(2):
                    for Q in range(2):
                        try:
                            model = SARIMAX(
                                train,
                                order=(p, d, q),
                                seasonal_order=(P, D, Q, 12),
                                enforce_stationarity=False,
                                enforce_invertibility=False
                            ).fit(disp=False, maxiter=200)
                            if model.aic < best_aic:
                                best_aic = model.aic
                                best_order = (p, d, q)
                                best_seasonal_order = (P, D, Q, 12)
                        except:
                            continue
        
        logger.info(f"Best SARIMA order: {best_order}, seasonal_order: {best_seasonal_order}")
        
        # Fit SARIMA and get residuals
        sarima_model = SARIMAX(
            train,
            order=best_order,
            seasonal_order=best_seasonal_order,
            enforce_stationarity=False,
            enforce_invertibility=False
        ).fit(disp=False, maxiter=200)
        sarima_forecast = sarima_model.forecast(steps=len(test))
        sarima_train_pred = sarima_model.fittedvalues
        residuals = train - sarima_train_pred
        residuals = residuals.rolling(window=3, min_periods=1).mean().dropna()  # Smooth residuals
        
        # Step 2: Prepare data for BiLSTM
        X_train_res = X_train.loc[residuals.index]
        y_train_res = residuals
        X_test_res = X_test.loc[y_test.index]
        
        y_scaler = RobustScaler()
        X_scaler = RobustScaler()
        y_train_res_scaled = y_scaler.fit_transform(y_train_res.values.reshape(-1, 1)).flatten()
        X_train_res_scaled = X_scaler.fit_transform(X_train_res)
        X_test_res_scaled = X_scaler.transform(X_test_res)
        
        seq_length = 12
        X_res_seq, y_res_seq = create_sequences(X_train_res_scaled, y_train_res_scaled, seq_length)
        if X_res_seq.size == 0:
            logger.error("Insufficient data for residual sequences. Returning SARIMA forecast.")
            return sarima_forecast, calculate_metrics(y_test, sarima_forecast, train)
        
        last_train_seq = X_train_res_scaled[-seq_length:]
        X_pred_input = np.concatenate([last_train_seq[:-1], X_test_res_scaled], axis=0)
        X_pred_seq, _ = create_sequences(X_pred_input, np.zeros(len(X_pred_input)), seq_length)
        
        if X_pred_seq.size == 0:
            logger.error("Failed to create prediction sequences. Returning SARIMA forecast.")
            return sarima_forecast, calculate_metrics(y_test, sarima_forecast, train)
        
        # Step 3: Optimize BiLSTM hyperparameters
        def objective(trial):
            hidden_size = trial.suggest_int("hidden_size", 32, 256)
            num_layers = trial.suggest_int("num_layers", 1, 4)
            dropout = trial.suggest_float("dropout", 0.1, 0.5)
            learning_rate = trial.suggest_float("learning_rate", 1e-4, 1e-2, log=True)
            batch_size = trial.suggest_categorical("batch_size", [16, 32, 64])
            
            dataset = TensorDataset(
                torch.FloatTensor(X_res_seq), 
                torch.FloatTensor(y_res_seq).unsqueeze(1)
            )
            train_size = int(0.8 * len(dataset))
            val_size = len(dataset) - train_size
            train_dataset, val_dataset = torch.utils.data.random_split(dataset, [train_size, val_size])
            
            train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
            val_loader = DataLoader(val_dataset, batch_size=batch_size)
            
            device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
            model = BiLSTMWithAttention(
                input_size=X_res_seq.shape[2],
                hidden_size=hidden_size,  # Use trial parameter
                num_layers=num_layers,    # Use trial parameter
                output_size=1,
                dropout=dropout           # Use trial parameter
            ).to(device)
            
            criterion = nn.MSELoss()
            optimizer = torch.optim.AdamW(model.parameters(), lr=learning_rate, weight_decay=1e-5)
            
            model.train()
            for epoch in range(20):
                for X_batch, y_batch in train_loader:
                    X_batch, y_batch = X_batch.to(device), y_batch.to(device)
                    optimizer.zero_grad()
                    y_pred = model(X_batch)
                    loss = criterion(y_pred, y_batch)
                    loss.backward()
                    optimizer.step()
            
            model.eval()
            val_loss = 0
            with torch.no_grad():
                for X_batch, y_batch in val_loader:
                    X_batch, y_batch = X_batch.to(device), y_batch.to(device)
                    y_pred = model(X_batch)
                    val_loss += criterion(y_pred, y_batch).item()
            
            return val_loss / len(val_loader)
        
        study = optuna.create_study(direction="minimize")
        study.optimize(objective, n_trials=50)
        best_params = study.best_params
        logger.info(f"Best BiLSTM params: {best_params}")
        
        # Step 4: Train BiLSTM
        dataset = TensorDataset(
            torch.FloatTensor(X_res_seq), 
            torch.FloatTensor(y_res_seq).unsqueeze(1)
        )
        train_loader = DataLoader(dataset, batch_size=best_params["batch_size"], shuffle=True)
        
        device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
        model = BiLSTMWithAttention(
            input_size=X_res_seq.shape[2],
            hidden_size=best_params["hidden_size"],
            num_layers=best_params["num_layers"],
            output_size=1,
            dropout=best_params["dropout"]
        ).to(device)
        
        criterion = nn.MSELoss()
        optimizer = torch.optim.AdamW(model.parameters(), lr=best_params["learning_rate"], weight_decay=1e-5)
        scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer, mode="min", factor=0.5, patience=5)
        
        best_loss = float("inf")
        patience_counter = 0
        checkpoint_path = f"{CONFIG['results_dir']}/SARIMA_BiLSTM_best.pth"
        
        model.train()
        for epoch in range(100):
            epoch_loss = 0
            for X_batch, y_batch in train_loader:
                X_batch, y_batch = X_batch.to(device), y_batch.to(device)
                optimizer.zero_grad()
                y_pred = model(X_batch)
                loss = criterion(y_pred, y_batch)
                loss.backward()
                optimizer.step()
                epoch_loss += loss.item()
            
            avg_loss = epoch_loss / len(train_loader)
            scheduler.step(avg_loss)
            
            if avg_loss < best_loss:
                best_loss = avg_loss
                torch.save(model.state_dict(), checkpoint_path)
                patience_counter = 0
            else:
                patience_counter += 1
                if patience_counter >= CONFIG['dl_patience']:
                    logger.info(f"Early stopping at epoch {epoch+1}")
                    break
            
            if (epoch + 1) % 10 == 0:
                logger.info(f"Epoch {epoch+1}/100, Loss: {avg_loss:.6f}")
        
        model.load_state_dict(torch.load(checkpoint_path))
        
        # Step 5: Forecast residuals
        model.eval()
        residual_forecast = []
        with torch.no_grad():
            for i in range(len(y_test)):
                if i + seq_length > len(X_pred_seq):
                    residual_forecast.append(0)
                    continue
                input_seq = X_pred_seq[i:i+1]
                input_tensor = torch.FloatTensor(input_seq).to(device)
                pred = model(input_tensor).cpu().numpy()
                residual_forecast.append(pred.item())
        
        residual_forecast = y_scaler.inverse_transform(np.array(residual_forecast).reshape(-1, 1)).flatten()
        residual_forecast = pd.Series(residual_forecast, index=y_test.index)
        
        # Step 6: Combine forecasts
        hybrid_forecast = sarima_forecast.reindex(y_test.index) + residual_forecast
        metrics = calculate_metrics(y_test, hybrid_forecast, train)
        logger.info(f"Hybrid SARIMA-BiLSTM metrics: RMSE={metrics['RMSE']:.4f}, MAPE={metrics['MAPE']:.4f}")
        
        return hybrid_forecast, metrics
    
    except Exception as e:
        logger.error(f"Error in Hybrid SARIMA-BiLSTM: {e}")
        return None, None
    

def run_hybrid_prophet_xgboost(train: pd.Series, test: pd.Series, 
                             X_train: pd.DataFrame, y_train: pd.Series, 
                             X_test: pd.DataFrame, y_test: pd.Series) -> Tuple[pd.Series, Dict[str, float]]:
    """Run enhanced Hybrid Prophet-XGBoost with feature selection and hyperparameter optimization."""
    logger.info("Running Enhanced Hybrid Prophet-XGBoost...")
    if not prophet_available:
        logger.warning("Prophet not available. Skipping Hybrid Prophet-XGBoost.")
        return None, None
    
    try:
        # Step 1: Run Prophet
        df_prophet = pd.DataFrame({"ds": train.index, "y": train.values})
        prophet_model = Prophet(
            yearly_seasonality=True,
            weekly_seasonality=False,
            daily_seasonality=False,
            changepoint_prior_scale=0.1
        )
        prophet_model.add_seasonality(name="monthly", period=30.5, fourier_order=10)
        prophet_model.fit(df_prophet)
        
        # Get in-sample and out-of-sample predictions
        prophet_train_pred = prophet_model.predict(df_prophet)
        prophet_train_pred = pd.Series(prophet_train_pred["yhat"].values, index=train.index)
        future = pd.DataFrame({"ds": test.index})
        prophet_forecast = prophet_model.predict(future)
        prophet_forecast = pd.Series(prophet_forecast["yhat"].values, index=test.index)
        
        # Calculate residuals
        residuals = train - prophet_train_pred
        residuals = residuals.dropna()
        
        # Step 2: Feature selection with RFE
        X_train_res = X_train.loc[residuals.index]
        y_train_res = residuals
        base_xgb = xgb.XGBRegressor(random_state=CONFIG['random_seed'])
        rfe = RFE(estimator=base_xgb, n_features_to_select=10)
        rfe.fit(X_train_res, y_train_res)
        selected_features = X_train_res.columns[rfe.support_].tolist()
        logger.info(f"Selected features for XGBoost: {selected_features}")
        
        X_train_res = X_train_res[selected_features]
        X_test_res = X_test[selected_features]
        
        # Step 3: Optimize XGBoost hyperparameters
        def objective(trial):
            params = {
                "n_estimators": trial.suggest_int("n_estimators", 50, 300),
                "max_depth": trial.suggest_int("max_depth", 3, 12),
                "learning_rate": trial.suggest_float("learning_rate", 0.01, 0.3, log=True),
                "subsample": trial.suggest_float("subsample", 0.6, 1.0),
                "colsample_bytree": trial.suggest_float("colsample_bytree", 0.6, 1.0),
                "random_state": CONFIG['random_seed']
            }
            model = xgb.XGBRegressor(**params)
            model.fit(X_train_res, y_train_res)
            pred = model.predict(X_test_res)
            return mean_squared_error(y_test.reindex(X_test_res.index), pred + prophet_forecast.reindex(X_test_res.index))
        
        if optuna_available:
            study = optuna.create_study(direction="minimize")
            study.optimize(objective, n_trials=CONFIG['optuna_trials'])
            best_params = study.best_params
            logger.info(f"Best XGBoost params: {best_params}")
        else:
            best_params = {
                "n_estimators": 100,
                "max_depth": 5,
                "learning_rate": 0.1,
                "subsample": 0.8,
                "colsample_bytree": 0.8,
                "random_state": CONFIG['random_seed']
            }
        
        # Step 4: Train XGBoost on residuals
        xgb_model = xgb.XGBRegressor(**best_params)
        xgb_model.fit(X_train_res, y_train_res)
        
        # Generate residual forecast
        residual_forecast = xgb_model.predict(X_test_res)
        residual_forecast = pd.Series(residual_forecast, index=y_test.index)
        
        # Combine forecasts
        hybrid_forecast = prophet_forecast.reindex(y_test.index) + residual_forecast
        
        # Calculate metrics
        metrics = calculate_metrics(y_test, hybrid_forecast, train)
        logger.info(f"Hybrid Prophet-XGBoost metrics: RMSE={metrics['RMSE']:.4f}, MAPE={metrics['MAPE']:.4f}")
        
        return hybrid_forecast, metrics
    except Exception as e:
        logger.error(f"Error in Hybrid Prophet-XGBoost: {e}")
        return None, None

# ================ MAIN FUNCTION ================

from joblib import Parallel, delayed

def run_model_wrapper(model_func, *args, **kwargs):
    try:
        return model_func(*args, **kwargs)
    except Exception as e:
        logger.error(f"Error in {model_func.__name__}: {e}")
        return None, None
    
def run_all_models(cpi_data_path: str, config: Dict = CONFIG):
    """Run all forecasting models on CPI data and save results."""
    start_time = time.time()
    logger.info("Starting CPI forecasting pipeline...")
    
    # Load data
    df = load_data(cpi_data_path)
    
    # Split into train and test (raw data)
    train_df, test_df = train_test_split_ts(df, test_months=config["test_months"])
    
    # Extract target series (raw)
    train_series = train_df["cpi"]
    test_series = test_df["cpi"]
    
    # Verify test set size
    if len(test_series) != config["test_months"]:
        logger.warning(f"Test set size ({len(test_series)}) does not match requested {config['test_months']} months.")
    
    # Prepare features and aligned targets for ML/DL models
    X_train, y_train = prepare_ml_features(train_df)
    X_test, y_test = prepare_ml_features(test_df)
    
    # Check if test features are valid
    if X_test.empty or y_test.empty:
        logger.warning("Test features or target is empty. Disabling ML and DL models.")
        config["enable_ml_models"] = False
        config["enable_dl_models"] = False
        config["enable_hybrid_models"] = False
        config["enable_ensemble_models"] = False
    
    # Dictionary to store all forecasts and metrics
    all_forecasts = {}
    all_metrics = {}
    train_forecasts = {}  # For stacking ensemble
    
    # Run traditional models (use raw train/test series)
    if config["enable_traditional_models"]:
        logger.info("Running traditional statistical models...")
        
        # Naive models
        naive_forecast, naive_metrics = run_naive_forecast(train_series, test_series)
        all_forecasts["Naive"] = naive_forecast
        all_metrics["Naive"] = naive_metrics
        
        seasonal_naive_forecast, seasonal_naive_metrics = run_seasonal_naive_forecast(train_series, test_series)
        all_forecasts["SeasonalNaive"] = seasonal_naive_forecast
        all_metrics["SeasonalNaive"] = seasonal_naive_metrics
        
        # Moving average
        ma_forecast, ma_metrics = run_average_forecast(train_series, test_series, window=12)
        all_forecasts["MovingAverage"] = ma_forecast
        all_metrics["MovingAverage"] = ma_metrics
        
        # Drift model
        drift_forecast, drift_metrics = run_drift_forecast(train_series, test_series)
        all_forecasts["Drift"] = drift_forecast
        all_metrics["Drift"] = drift_metrics
        
        # Exponential smoothing models
        ses_forecast, ses_metrics = run_ses_forecast(train_series, test_series)
        all_forecasts["SES"] = ses_forecast
        all_metrics["SES"] = ses_metrics
        
        holt_forecast, holt_metrics = run_holt_forecast(train_series, test_series)
        all_forecasts["Holt"] = holt_forecast
        all_metrics["Holt"] = holt_metrics
        
        hw_forecast, hw_metrics = run_holt_winters_forecast(train_series, test_series)
        all_forecasts["HoltWinters"] = hw_forecast
        all_metrics["HoltWinters"] = hw_metrics
        
        # ETS model
        ets_forecast, ets_metrics = run_ets_forecast(train_series, test_series)
        all_forecasts["ETS"] = ets_forecast
        all_metrics["ETS"] = ets_metrics
        
        # Theta model
        theta_forecast, theta_metrics = run_theta_forecast(train_series, test_series)
        all_forecasts["Theta"] = theta_forecast
        all_metrics["Theta"] = theta_metrics
        
        # ARIMA models
        arima_forecast, arima_metrics = run_arima_forecast(train_series, test_series)
        all_forecasts["ARIMA"] = arima_forecast
        all_metrics["ARIMA"] = arima_metrics
        
        sarima_forecast, sarima_metrics = run_sarima_forecast(train_series, test_series)
        all_forecasts["SARIMA"] = sarima_forecast
        all_metrics["SARIMA"] = sarima_metrics
        
        # Prophet
        if prophet_available:
            prophet_forecast, prophet_metrics = run_prophet_forecast(train_series, test_series)
            all_forecasts["Prophet"] = prophet_forecast
            all_metrics["Prophet"] = prophet_metrics
    
    # Run machine learning models (use prepared features/targets)
    if config["enable_ml_models"]:
        logger.info("Running machine learning models...")
        
        # Random Forest
        rf_forecast, rf_metrics = run_ml_model(
            "RandomForest", RandomForestRegressor, X_train, y_train, X_test, y_test, train_series
        )
        all_forecasts["RandomForest"] = rf_forecast
        all_metrics["RandomForest"] = rf_metrics
        
        # XGBoost
        xgb_forecast, xgb_metrics = run_ml_model(
            "XGBoost", xgb.XGBRegressor, X_train, y_train, X_test, y_test, train_series
        )
        all_forecasts["XGBoost"] = xgb_forecast
        all_metrics["XGBoost"] = xgb_metrics
        
        # LightGBM
        lgb_forecast, lgb_metrics = run_ml_model(
            "LightGBM", lgb.LGBMRegressor, X_train, y_train, X_test, y_test, train_series
        )
        all_forecasts["LightGBM"] = lgb_forecast
        all_metrics["LightGBM"] = lgb_metrics
        
        # SVR
        svr_forecast, svr_metrics = run_ml_model(
            "SVR", SVR, X_train, y_train, X_test, y_test, train_series
        )
        all_forecasts["SVR"] = svr_forecast
        all_metrics["SVR"] = svr_metrics
        
        # Gradient Boosting
        gb_forecast, gb_metrics = run_ml_model(
            "GradientBoosting", GradientBoostingRegressor, X_train, y_train, X_test, y_test, train_series
        )
        all_forecasts["GradientBoosting"] = gb_forecast
        all_metrics["GradientBoosting"] = gb_metrics
        
        # ElasticNet
        en_forecast, en_metrics = run_ml_model(
            "ElasticNet", ElasticNet, X_train, y_train, X_test, y_test, train_series
        )
        all_forecasts["ElasticNet"] = en_forecast
        all_metrics["ElasticNet"] = en_metrics
        
        # KNN
        knn_forecast, knn_metrics = run_ml_model(
            "KNN", KNeighborsRegressor, X_train, y_train, X_test, y_test, train_series
        )
        all_forecasts["KNN"] = knn_forecast
        all_metrics["KNN"] = knn_metrics

    # Run deep learning models
    if config["enable_dl_models"] and torch_available:
        logger.info("Running deep learning models...")
        
        seq_length = CONFIG['dl_sequence_length']
        X_train_seq, y_train_seq, X_pred_input, y_scaler, _ = prepare_dl_data(
            X_train, y_train, X_test, y_test, seq_length
        )
        
        if X_train_seq.size == 0 or y_train_seq.size == 0:
            logger.warning("Deep learning data preparation failed. Skipping DL models.")
            config["enable_dl_models"] = False
        else:
            # LSTM
            lstm_forecast, lstm_metrics = run_dl_model(
                "LSTM", X_train_seq, y_train_seq, X_pred_input, y_test, 
                y_scaler, seq_length, train_series
            )
            all_forecasts["LSTM"] = lstm_forecast
            all_metrics["LSTM"] = lstm_metrics
            
            # GRU
            gru_forecast, gru_metrics = run_dl_model(
                "GRU", X_train_seq, y_train_seq, X_pred_input, y_test, 
                y_scaler, seq_length, train_series
            )
            all_forecasts["GRU"] = gru_forecast
            all_metrics["GRU"] = gru_metrics
            
            # BiLSTM
            bilstm_forecast, bilstm_metrics = run_dl_model(
                "BiLSTM", X_train_seq, y_train_seq, X_pred_input, y_test, 
                y_scaler, seq_length, train_series
            )
            all_forecasts["BiLSTM"] = bilstm_forecast
            all_metrics["BiLSTM"] = bilstm_metrics
            
            # CNN-LSTM
            cnnlstm_forecast, cnnlstm_metrics = run_dl_model(
                "CNN-LSTM", X_train_seq, y_train_seq, X_pred_input, y_test, 
                y_scaler, seq_length, train_series
            )
            all_forecasts["CNN-LSTM"] = cnnlstm_forecast
            all_metrics["CNN-LSTM"] = cnnlstm_metrics
    
    # Generate in-sample forecasts for stacking ensemble
    if config["enable_ensemble_models"]:
        logger.info("Generating in-sample forecasts for stacking ensemble...")
        
        # Fit models on the entire training set and predict on it
        # Traditional models
        if config["enable_traditional_models"]:
            try:
                ets_model = ETSModel(train_series, error="add", trend="add", seasonal="add", seasonal_periods=12).fit(disp=False)
                train_forecasts["ETS"] = ets_model.fittedvalues
            except Exception as e: logger.warning(f"Failed to get ETS train forecast: {e}")
            try:
                d = check_stationarity(train_series)
                sarima_model = SARIMAX(train_series, order=(1, d, 1), seasonal_order=(1, 1, 1, 12), simple_differencing=True).fit(disp=False)
                train_forecasts["SARIMA"] = sarima_model.fittedvalues
            except Exception as e: logger.warning(f"Failed to get SARIMA train forecast: {e}")
            if prophet_available:
                try:
                    df_prophet = pd.DataFrame({"ds": train_series.index, "y": train_series.values})
                    prophet_model = Prophet(yearly_seasonality=True, weekly_seasonality=False, daily_seasonality=False).fit(df_prophet)
                    prophet_train_pred = prophet_model.predict(df_prophet)
                    train_forecasts["Prophet"] = pd.Series(prophet_train_pred["yhat"].values, index=train_series.index)
                except Exception as e: logger.warning(f"Failed to get Prophet train forecast: {e}")
        
        # ML models
        if config["enable_ml_models"]:
            try:
                rf_model = RandomForestRegressor(n_estimators=100, random_state=CONFIG['random_seed'])
                rf_model.fit(X_train, y_train)
                train_forecasts["RandomForest"] = pd.Series(rf_model.predict(X_train), index=y_train.index)
            except Exception as e: logger.warning(f"Failed to get RandomForest train forecast: {e}")
            try:
                xgb_model = xgb.XGBRegressor(n_estimators=100, random_state=CONFIG['random_seed'])
                xgb_model.fit(X_train, y_train)
                train_forecasts["XGBoost"] = pd.Series(xgb_model.predict(X_train), index=y_train.index)
            except Exception as e: logger.warning(f"Failed to get XGBoost train forecast: {e}")
            try:
                lgb_model = lgb.LGBMRegressor(n_estimators=100, random_state=CONFIG['random_seed'])
                lgb_model.fit(X_train, y_train)
                train_forecasts["LightGBM"] = pd.Series(lgb_model.predict(X_train), index=y_train.index)
            except Exception as e: logger.warning(f"Failed to get LightGBM train forecast: {e}")
    
    # Run hybrid and ensemble models
    if config["enable_hybrid_models"]:
        logger.info("Running hybrid models...")
        
        # SARIMA-LSTM hybrid
        if torch_available and config["enable_dl_models"]:
            sarima_lstm_forecast, sarima_lstm_metrics = run_hybrid_sarima_bilstm(
                train_series, test_series, X_train, y_train, X_test, y_test
            )
            all_forecasts["ARIMA-LSTM"] = sarima_lstm_forecast
            all_metrics["ARIMA-LSTM"] = sarima_lstm_metrics
        
        # Prophet-XGBoost hybrid
        if prophet_available:
            prophet_xgb_forecast, prophet_xgb_metrics = run_hybrid_prophet_xgboost(
                train_series, test_series, X_train, y_train, X_test, y_test
            )
            all_forecasts["Prophet-XGBoost"] = prophet_xgb_forecast
            all_metrics["Prophet-XGBoost"] = prophet_xgb_metrics
    
    if config["enable_ensemble_models"]:
        logger.info("Running ensemble models...")
        
        # Simple average ensemble
        simple_avg_forecast, simple_avg_metrics = run_simple_average_ensemble(
            all_forecasts, train_series, test_series
        )
        all_forecasts["SimpleAverage"] = simple_avg_forecast
        all_metrics["SimpleAverage"] = simple_avg_metrics
        
        # Weighted average ensemble
        weighted_avg_forecast, weighted_avg_metrics = run_weighted_average_ensemble(
            all_forecasts, all_metrics, train_series, test_series
        )
        all_forecasts["WeightedAverage"] = weighted_avg_forecast
        all_metrics["WeightedAverage"] = weighted_avg_metrics
        
        # Stacking ensemble
        stacking_forecast, stacking_metrics = run_stacking_ensemble(
            all_forecasts, train_forecasts, train_series, test_series
        )
        all_forecasts["Stacking"] = stacking_forecast
        all_metrics["Stacking"] = stacking_metrics
    
        # Dynamic Ensemble
        dynamic_ensemble_forecast, dynamic_ensemble_metrics = run_dynamic_ensemble(
            all_forecasts, train_forecasts, train_series, test_series
        )
        all_forecasts["DynamicEnsemble"] = dynamic_ensemble_forecast
        all_metrics["DynamicEnsemble"] = dynamic_ensemble_metrics

    # Convert metrics to DataFrame
    valid_metrics = {k: v for k, v in all_metrics.items() if v is not None}
    metrics_df = pd.DataFrame(valid_metrics).T
    
    # Sort models by RMSE
    if "RMSE" in metrics_df.columns:
        metrics_df = metrics_df.sort_values("RMSE")
    
    # Print top models
    logger.info("\nTop 10 models by RMSE:")
    logger.info(metrics_df.head(10)[["RMSE", "MAE", "MAPE"]])
    
    # Save results
    if config["save_forecasts"] or config["save_metrics"]:
        save_results(all_forecasts, metrics_df, config["results_dir"])
    
    # Generate plots
    if config["save_plots"]:
        # Plot individual forecasts
        for model_name, forecast in all_forecasts.items():
            if forecast is not None:
                plot_forecast(train_series, test_series, forecast, model_name, config["results_dir"])
        
        # Plot comparison of top models
        top_models = metrics_df.index[:10].tolist()
        top_forecasts = {model: all_forecasts[model] for model in top_models if model in all_forecasts and all_forecasts[model] is not None}
        plot_comparison(train_series, test_series, top_forecasts, metrics_df, config["results_dir"])
        
        # Plot metrics comparison
        plot_metrics_comparison(metrics_df, config["results_dir"])
    
    # Calculate total runtime
    total_time = time.time() - start_time
    logger.info(f"\nTotal runtime: {total_time:.2f} seconds ({total_time/60:.2f} minutes)")
    
    return all_forecasts, metrics_df

if __name__ == "__main__":
    # Run all models on CPI data
    cpi_data_path = "data/data.csv" 
    
    # Install prophet if needed
    if not prophet_available:
        try:
            import subprocess
            subprocess.check_call(["pip", "install", "prophet"])
            from prophet import Prophet
            prophet_available = True
            logger.info("Prophet installed successfully.")
        except Exception as e:
            logger.error(f"Failed to install Prophet: {e}")

    # Install optuna if needed
    if not optuna_available:
        try:
            import subprocess
            subprocess.check_call(["pip", "install", "optuna"])
            import optuna
            optuna_available = True
            optuna.logging.set_verbosity(optuna.logging.WARNING)
            logger.info("Optuna installed successfully.")
        except Exception as e:
            logger.error(f"Failed to install Optuna: {e}")
            
    # Install torch if needed
    if not torch_available:
        try:
            import subprocess
            subprocess.check_call(["pip", "install", "torch"])
            import torch
            import torch.nn as nn
            from torch.utils.data import TensorDataset, DataLoader
            torch_available = True
            logger.info("PyTorch installed successfully.")
        except Exception as e:
            logger.error(f"Failed to install PyTorch: {e}")
            
    # Run the pipeline
    forecasts, metrics = run_all_models(cpi_data_path)


2025-05-26 21:42:48,178 - INFO - Starting CPI forecasting pipeline...
2025-05-26 21:42:48,178 - INFO - Loading data from data/data.csv
2025-05-26 21:42:48,185 - INFO - Data loaded. Shape: (360, 3)
2025-05-26 21:42:48,186 - INFO - Date range: 1995-01-01 00:00:00 to 2024-12-01 00:00:00
2025-05-26 21:42:48,188 - INFO - Train set: (348, 3), Test set: (12, 3)
2025-05-26 21:42:48,189 - INFO - Train period: 1995-01-01 00:00:00 to 2023-12-01 00:00:00
2025-05-26 21:42:48,190 - INFO - Test period: 2024-01-01 00:00:00 to 2024-12-01 00:00:00
2025-05-26 21:42:48,191 - INFO - Preparing features for ML models...
2025-05-26 21:42:48,192 - INFO - Creating features with enhanced feature engineering...
2025-05-26 21:42:48,209 - INFO - Features created. Shape: (348, 37)
2025-05-26 21:42:48,211 - INFO - ML features prepared. X shape: (348, 37), y shape: (348,)
2025-05-26 21:42:48,212 - INFO - Preparing features for ML models...
2025-05-26 21:42:48,212 - INFO - Creating features with enhanced feature engine

[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.000174 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 2117
[LightGBM] [Info] Number of data points in the train set: 278, number of used features: 37
[LightGBM] [Info] Start training from score 100.535827
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain,

2025-05-26 21:44:38,266 - INFO - Best LightGBM params: {'n_estimators': 147, 'max_depth': 4, 'learning_rate': 0.1740291045306993, 'num_leaves': 82, 'subsample': 0.6188868704285965, 'colsample_bytree': 0.844687711378575}
2025-05-26 21:44:38,305 - INFO - LightGBM Forecast metrics: RMSE=0.1228, MAPE=0.0910
2025-05-26 21:44:38,306 - INFO - Running SVR model...
2025-05-26 21:44:38,315 - INFO - Optimizing hyperparameters for SVR with Optuna...


[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No f

2025-05-26 21:44:38,511 - INFO - Best SVR params: {'C': 1.5412341108937173, 'epsilon': 0.018632265993693616, 'gamma': 'scale'}
2025-05-26 21:44:38,523 - INFO - SVR Forecast metrics: RMSE=0.6616, MAPE=0.6234
2025-05-26 21:44:38,524 - INFO - Running GradientBoosting model...
2025-05-26 21:44:38,526 - INFO - Optimizing hyperparameters for GradientBoosting with Optuna...
2025-05-26 21:45:05,136 - INFO - Best GradientBoosting params: {'n_estimators': 270, 'max_depth': 4, 'learning_rate': 0.08322472513956597, 'subsample': 0.9992949011642009}
2025-05-26 21:45:07,582 - INFO - GradientBoosting Forecast metrics: RMSE=0.1345, MAPE=0.0608
2025-05-26 21:45:07,584 - INFO - Running ElasticNet model...
2025-05-26 21:45:07,593 - INFO - Optimizing hyperparameters for ElasticNet with Optuna...
2025-05-26 21:45:07,967 - INFO - Best ElasticNet params: {'alpha': 0.0001012668208754566, 'l1_ratio': 0.8310681721690936}
2025-05-26 21:45:07,987 - INFO - ElasticNet Forecast metrics: RMSE=0.1801, MAPE=0.0776
2025-

[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.000473 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 2712
[LightGBM] [Info] Number of data points in the train set: 348, number of used features: 37
[LightGBM] [Info] Start training from score 100.476494
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain,

2025-05-27 01:02:00,819 - INFO - Best SARIMA order: (1, 0, 0), seasonal_order: (0, 1, 1, 12)
2025-05-27 01:02:01,071 - INFO - Creating sequences with length 12...
2025-05-27 01:02:01,073 - INFO - Sequences created. X shape: (336, 12, 37), y shape: (336,)
2025-05-27 01:02:01,075 - INFO - Creating sequences with length 12...
2025-05-27 01:02:01,076 - INFO - Sequences created. X shape: (11, 12, 37), y shape: (11,)
2025-05-27 01:10:20,531 - INFO - Best BiLSTM params: {'hidden_size': 129, 'num_layers': 4, 'dropout': 0.37913913139300776, 'learning_rate': 0.0025645589918778084, 'batch_size': 32}
2025-05-27 01:10:30,850 - INFO - Epoch 10/100, Loss: 13.771638
2025-05-27 01:10:41,278 - INFO - Epoch 20/100, Loss: 13.269469
2025-05-27 01:10:47,279 - INFO - Early stopping at epoch 26
2025-05-27 01:10:47,289 - INFO - Hybrid SARIMA-BiLSTM metrics: RMSE=0.1981, MAPE=0.1801
2025-05-27 01:10:47,294 - INFO - Running Enhanced Hybrid Prophet-XGBoost...
2025-05-27 01:10:47,306 - ERROR - Error in Hybrid Prop